# BlackSheep Cookbook Exploration

The Black Sheep Analysis allows researchers to find trends in abnormal protein enrichment among patients in CPTAC datasets. In this Cookbook, we will go through the steps needed to perform a full Black Sheep Analysis.

### Step 1a: Import Dependencies
First, import the necessary dependencies and load cptac data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cptac
import binarization_functions as bf
import blackSheepCPTACmoduleCopy as blsh
import gseapy as gp
from gseapy.plot import barplot, heatmap, dotplot
import json
import requests
import ujson
import simdjson

## Step 1b: Load Data and Choose Omics Table
For this analysis, we will be looking at results across the proteomics, transcriptomics, and phosphoproteomics tables.

In [2]:
cptac.download(dataset='renalccrcc')

All files already downloaded and correct.


True

In [3]:
re = cptac.RenalCcrcc()
proteomics = re.get_proteomics()
#transcriptomics = re.get_transcriptomics()
clinical = re.get_clinical()

## Step 2: Determine what attributes you would like to A/B test. 
For this analysis, we will iteratively go through the various columns in the clinical dataset, to determine if any of them have trends within them for protein enrichment.

In [4]:
#Data cleaning. We want to get rid of non_clear cell carcinoma, 
#as it throws off our analysis, and isn't what we are studying
non_clear_cell_filter = clinical['histologic_type'] == 'non-Clear Cell renal cell carcinoma'
clinical = clinical[~non_clear_cell_filter]

In [5]:
#We also need to get rid of those samples in the datasets we plan to use. In this case, Proteomics.
rows_to_drop = []

for i, item in enumerate(proteomics.index):
    if item not in list(clinical.index):
        rows_to_drop.append(item)
proteomics.drop(rows_to_drop)

print("Rows Dropped:")
print(rows_to_drop)

Rows Dropped:
['S011', 'S097', 'S104', 'S107', 'S115', 'S119', 'S123']


In [6]:
#Create a copy of the original Clinical DataFrame
annotations = pd.DataFrame(clinical.copy())

In [7]:
#Convert columns that should be numeric to numeric
annotations['age'] = annotations['age'].replace('>=90', 90)
annotations['age'] = pd.to_numeric(annotations['age'])

annotations['age_started_smoking'] = annotations['age_started_smoking'].replace('Unknown', np.nan)
annotations['age_started_smoking'] = pd.to_numeric(annotations['age_started_smoking'])

annotations['age_stopped_smoking'] = annotations['age_stopped_smoking'].replace('Unknown', np.nan)
annotations['age_stopped_smoking'] = pd.to_numeric(annotations['age_stopped_smoking'])

annotations['how_many_cigarettes_usually_smoke_on_the_days_participant_smoked'] = annotations['how_many_cigarettes_usually_smoke_on_the_days_participant_smoked'].replace('Unknown', np.nan)
annotations['how_many_cigarettes_usually_smoke_on_the_days_participant_smoked'] = pd.to_numeric(annotations['how_many_cigarettes_usually_smoke_on_the_days_participant_smoked'])

In [8]:
#Drop irrelevant columns
irrelevant_cols = ['Patient_ID', 'height_in_cm', 'weight_in_kg', 'race', 'ethnicity', 
                   'ethnicity_self_identified','tumor_site_other', 'sarcomatoid_features', 
                   'pathologic_findings_in_nonneoplastic_kidney_specify', 
                   'other_immunohistochemistry_specify', 'other_testing_performed', 
                   'other_coagulopathies', 'exposure_to_secondhand_smoke', 
                   'tumor_tissue_collection_clamps_used','tumor_tissue_collection_frozen_with_OCT', 
                   'preoperative_IV_sedation_administered',  'preoperative_IV_opiates_administered', 
                   'preoperative_IV_antiemetics_administered','preoperative_IV_anti_acids_administered', 
                   'other_preoperative_IV_medications_administered', 'platelets',
                   'local_anesthesia_agents_administered', 'regional_anesthesia_agents_administered', 
                   'IV_anesthesia_agents_administered', 'IV_narcotic_opiate_agents_administered', 
                   'IV_muscle_relaxants_administered', 'inhalation_anesthesia_agents_administered', 
                   'additional_anesthesia', 'other_medications_administered_during_surgery', 
                   'surgical_procedure', 'epochs_of_oxygen_desaturation', 'time_specimen_left_OR', 
                   'first_participant_temperature_recorded_in_OR', 'first_temperature_recorded_in_OR_units', 
                   'first_temperature_recorded_in_OR_celsius', 'time_of_first_temperature', 
                   'first_temperature_recorded_in_OR_fahrenheit', 'second_participant_temperature_recorded_in_OR', 
                   'second_temperature_recorded_in_OR_units','second_temperature_recorded_in_OR_celsius', 
                   'second_temperature_recorded_in_OR_fahrenheit', 'time_of_second_temperature', 
                   'Karnofsky_score_at_12months_follow_up', 'Karnofsky_score_at_24months_follow_up', 
                   'performance_status_scale_timing_at_12months_follow_up', 
                   'performance_status_scale_timing_at_24months_follow_up', 
                   'adjuvant_postoperative_radiation_therapy_at_12months_follow_up', 
                   'adjuvant_postoperative_radiation_therapy_at_24months_follow_up',  
                   'adjuvant_postoperative_immunological_therapy_at_12months_follow_up', 
                   'adjuvant_postoperative_immunological_therapy_at_24months_follow_up', 
                   'adjuvant_postoperative_pharmaceutical_therapy_at_12months_follow_up', 
                   'adjuvant_postoperative_pharmaceutical_therapy_at_24months_follow_up', 
                   'ECOG_score_at_12months_follow_up', 'ECOG_score_at_24months_follow_up', 
                   'specimen_id', 'slide_id', 'Presence_sarcomatoid_features', 
                   'non_cellular_component_description', 'meets_required_criteria', 
                   'histologic_type_of_normal_tissue', 'findings_not_consistent_with_local_pathology_report', 
                   'minutes_clamp_1_to_collection', 'minutes_clamp_2_to_collection', 
                   'consistent_with_diagnostic_report', 'patient_medications']

annotations = annotations.drop(irrelevant_cols, axis=1)

#Columns we should either drop, or be generally skeptical of in our analysis
questionable_cols = ['specify_distant_metastasis_documented_sites', 
                     'erythrocyte_sedimentation_rate', 
                     'renal_vein_or_vena_cava_thrombosis', 
                     'CO2_level_recorded_at_time_closest_to_organ_excision']

annotations = annotations.drop(questionable_cols, axis=1)

In [9]:
#Determine which columns are binary and which aren't
binary_cols = []
non_binary_cols = []

for col in annotations.columns:
    #this is the number of datapoints in a column
    if annotations[col].describe()[0] <= 10:
        annotations = annotations.drop(col, axis=1)
    elif len(annotations[col].value_counts()) == 2:
        binary_cols.append(col)
    elif len(annotations[col].value_counts()) > 2:
        if np.issubdtype(annotations[col].dtype, np.number):
            mean = annotations[col].mean()
            annotations[col]= bf.binarizeCutOff(annotations, col, mean, 
                                                "Above_Mean("+str(round(mean, 2))+")", 
                                                "Below_Mean("+str(round(mean, 2))+")")
        else:
            non_binary_cols.append(col)
    else:
        annotations = annotations.drop(col, axis=1)

'''numeric_non_bin = []
categorical_non_bin = []

for item in non_binary_cols:
    if np.issubdtype(annotations[item].dtype, np.number):
        numeric_non_bin.append(item)
        mean = annotations[item].mean()
        annotations[item]= bf.binarizeCutOff(annotations, item, mean, 
                                             "Above_Mean("+str(round(mean, 2))+")", 
                                             "Below_Mean("+str(round(mean, 2))+")")
    else:
        categorical_non_bin.append(item)
'''

'numeric_non_bin = []\ncategorical_non_bin = []\n\nfor item in non_binary_cols:\n    if np.issubdtype(annotations[item].dtype, np.number):\n        numeric_non_bin.append(item)\n        mean = annotations[item].mean()\n        annotations[item]= bf.binarizeCutOff(annotations, item, mean, \n                                             "Above_Mean("+str(round(mean, 2))+")", \n                                             "Below_Mean("+str(round(mean, 2))+")")\n    else:\n        categorical_non_bin.append(item)\n'

## Step 2a: Binarize column values

In [10]:
tumor_site_map = {'Other':'Other',
                  'Upper pole':'Pole',
                  'Middle':'Pole',
                  'Lower pole':'Pole'}

annotations['tumor_site'] = bf.binarizeCategorical(annotations,
                                                   'tumor_site',
                                                   tumor_site_map)

In [11]:
histologic_grade_map = {'G1':'G1or2',
                        'G2':'G1or2',
                        'G3':'G3or4',
                        'G4':'G3or4'}

annotations['histologic_grade'] = bf.binarizeCategorical(annotations, 
                                                         'histologic_grade',
                                                         histologic_grade_map)

In [12]:
tumor_stage_pathological_map = {'Stage I':'StageIorII', 
                                'Stage II':'StageIorII', 
                                'Stage III':'StageIIIorIV',
                                'Stage V':'StageIIIorV'}

annotations['tumor_stage_pathological'] = bf.binarizeCategorical(annotations,
                                                                 'tumor_stage_pathological',
                                                                 tumor_stage_pathological_map)

In [13]:
pathologic_staging_map = {'pT3a':'pT3or4', 
                          'pT1a':'pT1or2', 
                          'pT1b':'pT1or2', 
                          'pT2a':'pT1or2', 
                          'pT2b':'pT1or2', 
                          'pT3':'pT3or4', 
                          'pT3b':'pT3or4', 
                          'pT4':'pT3or4'}

annotations['pathologic_staging_primary_tumor_pT'] = bf.binarizeCategorical(annotations,
                                                                            'pathologic_staging_primary_tumor_pT',
                                                                            pathologic_staging_map)

In [14]:
pathologic_lymph_map = {'pNX':'pNX', 
                        'pN0':'pN0or1', 
                        'pN1':'pN0or1'}

annotations['pathologic_staging_regional_lymph_nodes_pN'] = bf.binarizeCategorical(annotations,
                                                                                   'pathologic_staging_regional_lymph_nodes_pN',
                                                                                   pathologic_lymph_map)

In [15]:
pathologic_metastasis_map = {'Staging Incomplete':np.nan,
                             'No pathologic evidence of distant metastasis':'No',
                             'pM1':'pM1'}

annotations['pathologic_staging_distant_metastasis_pM'] = bf.binarizeCategorical(annotations,
                                                                                 'pathologic_staging_distant_metastasis_pM',
                                                                                 pathologic_metastasis_map)

In [16]:
annotations['clinical_staging_distant_metastasis_cM'] = annotations['clinical_staging_distant_metastasis_cM'].replace('Staging Incomplete', np.nan)

In [17]:
serum_calcium_map = {'Not evaluated':np.nan,
                     'Unknown':np.nan,
                     'Normal':'Normal',
                     'Low':'Abnormal',
                     'Elevated':'Abnormal'}

annotations['serum_calcium'] = bf.binarizeCategorical(annotations,
                                                      'serum_calcium',
                                                      serum_calcium_map)

In [18]:
hemoglobin_map = {'Not evaluated':np.nan,
                  'Unknown':np.nan,
                  'Normal':'Normal',
                  'Low':'Abnormal',
                  'Elevated':'Abnormal'}

annotations['hemoglobin'] = bf.binarizeCategorical(annotations,
                                                   'hemoglobin',
                                                   hemoglobin_map)

In [19]:
white_cell_map = {'Not evaluated':np.nan,
                  'Unknown':np.nan,
                  'Normal':'Normal',
                  'Low':'Abnormal',
                  'Elevated':'Abnormal'}

annotations['white_cell_count'] = bf.binarizeCategorical(annotations,
                                                         'white_cell_count',
                                                         white_cell_map)

In [20]:
country_map = {'United States':'US', 
               'Poland':'Other',
               'Other':'Other', 
               'Ukraine':'Other',
               'Vietnam':'Other'}

annotations['participant_country_of_origin'] = bf.binarizeCategorical(annotations,
                                                                      'participant_country_of_origin',
                                                                      country_map)

In [21]:
annotations['margin_status'] = annotations['margin_status'].replace('Cannot be assessed', np.nan)

In [22]:
annotations['residual_tumor'] = annotations['residual_tumor'].replace('RX:Presence of residual tumor cannot be assessed', np.nan)

In [23]:
annotations['other_immunohistochemistry_performed'] = annotations['other_immunohistochemistry_performed'].replace('Unknown', np.nan)

In [24]:
alcohol_map = {'Lifelong non-drinker':'Non-drinker',
               'Alcohol consumption equal to or less than 2 drinks per day for men and 1 drink or less per day for women':'Previous_or_Current_Drinker',
               'Alcohol consumption history not available':np.nan,
               'Alcohol consumption more than 2 drinks per day for men and more than 1 drink per day for women':'Previous_or_Current_Drinker',
               'Consumed alcohol in the past, but currently a non-drinker':'Previous_or_Current_Drinker'}

annotations['alcohol_consumption'] = bf.binarizeCategorical(annotations, 
                                                            'alcohol_consumption', 
                                                            alcohol_map)

In [25]:
tobacco_map = {'Lifelong non-smoker: Less than 100 cigarettes smoked in lifetime':'Non-smoker', 
               'Smoking history not available':np.nan, 
               'Current reformed smoker within past 15 years':'Non-smoker', 
               'Current smoker: Includes daily and non-daily smokers':'Current Smoker', 
               'Current reformed smoker, more than 15 years':'Non-smoker'}

annotations['tobacco_smoking_history'] = bf.binarizeCategorical(annotations, 
                                                                'tobacco_smoking_history', 
                                                                tobacco_map)

In [26]:
annotations['tumor_status_at_12months_follow_up'] = annotations['tumor_status_at_12months_follow_up'].replace('Unknown', np.nan)

In [27]:
annotations['tumor_status_at_24months_follow_up'] = annotations['tumor_status_at_24months_follow_up'].replace('Unknown', np.nan)

In [28]:
success_map1 = {'Complete Remission':'Complete_Remission', 
                'Not Applicable':np.nan, 'Unknown':np.nan, 
                'Persistent Disease':'Persistent_Disease_or_Deceased', 
                'Patient Deceased':'Persistent_Disease_or_Deceased'}

annotations['success_of_initial_treatment_at_12months_follow_up'] = bf.binarizeCategorical(annotations, 
                                                                                           'success_of_initial_treatment_at_12months_follow_up', 
                                                                                           success_map1)

In [29]:
success_map2 = {'Complete Remission':'Complete_Remission', 
                'Not Applicable':np.nan, 'Unknown':np.nan, 
                'Persistent Disease':'Persistent_Disease'}

annotations['success_of_initial_treatment_at_24months_follow_up'] = bf.binarizeCategorical(annotations, 
                                                                                           'success_of_initial_treatment_at_24months_follow_up', 
                                                                                           success_map1)

In [30]:
success_map3 = {'Complete Remission':'Complete_Remission', 
                'Not Applicable':np.nan, 'Unknown':np.nan, 
                'Persistent Disease':'Persistent_Disease_or_Deceased', 
                'Patient Deceased':'Persistent_Disease_or_Deceased'}

annotations['success_of_treatment_at_12months_follow_up'] = bf.binarizeCategorical(annotations,
                                                                                   'success_of_treatment_at_12months_follow_up',
                                                                                   success_map3)

In [31]:
success_map4 = {'Complete Remission':'Complete_Remission', 
                'Not Applicable':np.nan, 'Unknown':np.nan, 
                'Persistent Disease':'Persistent_Disease'}

annotations['success_of_treatment_at_24months_follow_up'] = bf.binarizeCategorical(annotations,
                                                                                   'success_of_treatment_at_24months_follow_up',
                                                                                   success_map4)

In [32]:
annotations['new_tumor_event_after_initial_treatment_at_12months_follow_up'] = annotations['new_tumor_event_after_initial_treatment_at_12months_follow_up'].replace('Unknown', np.nan)

In [33]:
annotations['new_tumor_event_after_initial_treatment_at_24months_follow_up'] = annotations['new_tumor_event_after_initial_treatment_at_24months_follow_up'].replace('Unknown', np.nan)

## Step 3: Perform outliers analysis

In [34]:
outliers_up = blsh.make_outliers_table(proteomics, iqrs=1.5, 
                                       up_or_down='up', 
                                       aggregate=False, 
                                       frac_table=False)   

In [35]:
'''outliers_down = blsh.make_outliers_table(proteomics, iqrs=1.5, 
                                         up_or_down='down', 
                                         aggregate=False, 
                                         frac_table=False)     
'''

"outliers_down = blsh.make_outliers_table(proteomics, iqrs=1.5, \n                                         up_or_down='down', \n                                         aggregate=False, \n                                         frac_table=False)     \n"

In [36]:
'''
#Get only the outliers for Up-regulation. The second half of index is the _notOutliers, which we don't need
length = int(len(outliers_up.index) / 2)
only_outliers_up = outliers_up[:length]
outlier_up_dict = {}
for i in range(len(only_outliers_up)):
    key = list(proteomics.index)[i]
    my_filter = only_outliers_up.iloc[i] == 1.0
    value = list(only_outliers_up.iloc[i][my_filter].index)
    outlier_up_dict[key] = value
    print('Key:', key)
    print('{} outliers'.format(len(value)))
    #print('Value:', value)
    print('\n')
'''

"\n#Get only the outliers for Up-regulation. The second half of index is the _notOutliers, which we don't need\nlength = int(len(outliers_up.index) / 2)\nonly_outliers_up = outliers_up[:length]\noutlier_up_dict = {}\nfor i in range(len(only_outliers_up)):\n    key = list(proteomics.index)[i]\n    my_filter = only_outliers_up.iloc[i] == 1.0\n    value = list(only_outliers_up.iloc[i][my_filter].index)\n    outlier_up_dict[key] = value\n    print('Key:', key)\n    print('{} outliers'.format(len(value)))\n    #print('Value:', value)\n    print('\n')\n"

In [37]:
'''
#Get only the outliers for down-regulation. The second half of index is the _notOutliers, which we don't need
length = int(len(outliers_down.index) / 2)
only_down_outliers = outliers_down[:length]
outlier_down_dict = {}
for i in range(len(only_down_outliers)):
    key = list(proteomics.index)[i]
    my_filter = only_down_outliers.iloc[i] == 1.0
    value = list(only_down_outliers.iloc[i][my_filter].index)
    outlier_down_dict[key] = value
    print('Key:', key)
    #print('{} outliers'.format(len(value)))
    print('Value:', value)
    print('\n')
'''

"\n#Get only the outliers for down-regulation. The second half of index is the _notOutliers, which we don't need\nlength = int(len(outliers_down.index) / 2)\nonly_down_outliers = outliers_down[:length]\noutlier_down_dict = {}\nfor i in range(len(only_down_outliers)):\n    key = list(proteomics.index)[i]\n    my_filter = only_down_outliers.iloc[i] == 1.0\n    value = list(only_down_outliers.iloc[i][my_filter].index)\n    outlier_down_dict[key] = value\n    print('Key:', key)\n    #print('{} outliers'.format(len(value)))\n    print('Value:', value)\n    print('\n')\n"

## Step 4: Wrap your A/B test into the outliers analysis, and create a table
Note: compare_groups_outliers will break if there are duplicate gene names. Be sure to use the 
renameDuplicateColumns function if there are indexing errors

In [38]:
bf.renameDuplicateColumns(outliers_up)
#bf.renameDuplicateColumns(outliers_down)

results_up = blsh.compare_groups_outliers(outliers_up, 
                                          annotations)

No rows had outliers in at least 0.3 of Sample_Tumor_Normal Tumor samples
Testing 1 rows for enrichment in Sample_Tumor_Normal Normal samples
No rows had outliers in at least 0.3 of tumor/normal TN samples
Testing 2 rows for enrichment in tumor/normal T samples
No rows had outliers in at least 0.3 of gender Male samples
Testing 9 rows for enrichment in gender Female samples
No rows had outliers in at least 0.3 of age Above_Mean(60.79) samples
Testing 1 rows for enrichment in age Below_Mean(60.79) samples
No rows had outliers in at least 0.3 of height_in_inch Above_Mean(67.96) samples
Testing 2 rows for enrichment in height_in_inch Below_Mean(67.96) samples
No rows had outliers in at least 0.3 of weight_in_lb Below_Mean(209.02) samples
Testing 3 rows for enrichment in weight_in_lb Above_Mean(209.02) samples
No rows had outliers in at least 0.3 of BMI Below_Mean(31.77) samples
No rows had outliers in at least 0.3 of BMI Above_Mean(31.77) samples
No rows had outliers in at least 0.3 of tu

Testing 20 rows for enrichment in new_tumor_event_after_initial_treatment_at_12months_follow_up Yes samples
No rows had outliers in at least 0.3 of new_tumor_event_after_initial_treatment_at_24months_follow_up No samples
Testing 57 rows for enrichment in new_tumor_event_after_initial_treatment_at_24months_follow_up Yes samples
No rows had outliers in at least 0.3 of days_from_initial_diagnosis_to_last_contact_at_12months_follow_up Above_Mean(296.27) samples
Testing 1 rows for enrichment in days_from_initial_diagnosis_to_last_contact_at_12months_follow_up Below_Mean(296.27) samples
No rows had outliers in at least 0.3 of days_from_initial_diagnosis_to_last_contact_at_24months_follow_up Above_Mean(628.4) samples
Testing 8 rows for enrichment in days_from_initial_diagnosis_to_last_contact_at_24months_follow_up Below_Mean(628.4) samples
No rows had outliers in at least 0.3 of percent_tumor_surface_area Above_Mean(95.58) samples
No rows had outliers in at least 0.3 of percent_tumor_surface_

In [39]:
'''results_down = blsh.compare_groups_outliers(outliers_down, 
                                            annotations)
                            '''

'results_down = blsh.compare_groups_outliers(outliers_down, \n                                            annotations)\n                            '

Many of the output values from compare_group_outliers are NaN, so here we will get rid of the NaN values for visualization purposes.

In [40]:
results_up = results_up.dropna(axis=0, how='all')
#results_down = results_down.dropna(axis=0, how='all')

## Step 5: Visualize these enrichments

First, we will determine which clinical attributes have significant enrichments, and save them to a list for both up-regulated outliers, and down-regulated outliers.

In [41]:
#Drop Columns with less than 4 significant up-regulated enrichments
print("TESTING FOR UP-REGULATION:")
sig_cols_up = []
for col in results_up.columns:
    sig_col = bf.significantEnrichments(results_up, col)
    if sig_col is not None and len(sig_col) >= 4:
        sig_cols_up.append(sig_col)
        print(sig_col.head())
    else:
        results_up = results_up.drop(col, axis=1)

TESTING FOR UP-REGULATION:
1 significant protein enrichment in Sample_Tumor_Normal_Normal:

1 significant protein enrichment in tumor/normal_T:

3 significant protein enrichments in gender_Female

2 significant protein enrichments in weight_in_lb_Above_Mean(209.02)

8 significant protein enrichments in tumor_site_Other

         tumor_site_Other_P_values
ACBD3                     0.011011
AVL9                      0.002727
COG1                      0.002727
COG4                      0.022867
GOLPH3L                   0.002565
5 significant protein enrichments in tumor_size_in_cm_Above_Mean(6.51)

         tumor_size_in_cm_Above_Mean(6.51)_P_values
ACBD3                                      0.011284
GOLPH3L                                    0.005659
GSDME                                      0.004258
IGF2BP3                                    0.004258
TXNRD1                                     0.004258
16 significant protein enrichments in tumor_focality_Multifocal

       tumor_focali


2 significant protein enrichments in days_from_initial_diagnosis_to_last_contact_at_24months_follow_up_Below_Mean(628.4)

1 significant protein enrichment in percent_tumor_nuclei_Below_Mean(85.87):

1 significant protein enrichment in percent_tumor_nuclei_Above_Mean(85.87):

7 significant protein enrichments in percent_necrotic_surface_area_Above_Mean(5.21)

        percent_necrotic_surface_area_Above_Mean(5.21)_P_values
CPN1                                             0.005534      
F5                                               0.005534      
GNPTG                                            0.005534      
MRFAP1                                           0.007096      
SCYL3                                            0.011334      


In [42]:
'''#Drop Columns with less than 4 significant down-regulated enrichments
print("TESTING FOR DOWN-REGULATION:")
sig_cols_down = []
for col in results_down.columns:
    sig_col = bf.significantEnrichments(results_down, col)
    if sig_col is not None and len(sig_col) >= 4:
        sig_cols_down.append(sig_col)
        print(sig_col.head())
    else:
        results_down = results_down.drop(col, axis=1)
'''

'#Drop Columns with less than 4 significant down-regulated enrichments\nprint("TESTING FOR DOWN-REGULATION:")\nsig_cols_down = []\nfor col in results_down.columns:\n    sig_col = bf.significantEnrichments(results_down, col)\n    if sig_col is not None and len(sig_col) >= 4:\n        sig_cols_down.append(sig_col)\n        print(sig_col.head())\n    else:\n        results_down = results_down.drop(col, axis=1)\n'

Then we will plot the results, to visualize what genes are significantly enriched for the clinical attributes.

In [43]:
'''ax1 = plt.axes()
sns.heatmap(results_up, ax = ax1)
ax1.set_title('Up-Regulated Protein Enrichments for Renal Cancer Tumors')
plt.show()

ax2 = plt.axes()
sns.heatmap(results_down, ax = ax2)
ax2.set_title('Down-Regulated Protein Enrichments for Renal Cancer Tumors')
plt.show()
'''

"ax1 = plt.axes()\nsns.heatmap(results_up, ax = ax1)\nax1.set_title('Up-Regulated Protein Enrichments for Renal Cancer Tumors')\nplt.show()\n\nax2 = plt.axes()\nsns.heatmap(results_down, ax = ax2)\nax2.set_title('Down-Regulated Protein Enrichments for Renal Cancer Tumors')\nplt.show()\n"

## Step 6: Determine significant enrichments, and link with cancer drug database.

In [44]:
'''for col in sig_cols_up:
    print(col.columns[0])
    print(list(col.index))
    print('\n')
'''

"for col in sig_cols_up:\n    print(col.columns[0])\n    print(list(col.index))\n    print('\n')\n"

In [45]:
'''for col in sig_cols_down:
    print(col.columns[0])
    print(list(col.index))
    print('\n')
'''

"for col in sig_cols_down:\n    print(col.columns[0])\n    print(list(col.index))\n    print('\n')\n"

In [46]:
'''for col in sig_cols_up:
    col_name = col.columns[0]
    col_name = "_".join((col_name.split(" ")))
    print(col_name)
    gene_name_list = list(col.index)
    enrichment = gp.enrichr(gene_list = gene_name_list, 
                            description=col_name, 
                            gene_sets='KEGG_2019_Human', 
                            outdir='test/renal')
    print(enrichment.res2d)
    barplot(enrichment.res2d, title=col_name, ofname=col_name)
'''

'for col in sig_cols_up:\n    col_name = col.columns[0]\n    col_name = "_".join((col_name.split(" ")))\n    print(col_name)\n    gene_name_list = list(col.index)\n    enrichment = gp.enrichr(gene_list = gene_name_list, \n                            description=col_name, \n                            gene_sets=\'KEGG_2019_Human\', \n                            outdir=\'test/renal\')\n    print(enrichment.res2d)\n    barplot(enrichment.res2d, title=col_name, ofname=col_name)\n'

In [47]:
'''for col in sig_cols_down:
    col_name = col.columns[0]
    col_name = "_".join((col_name.split(" ")))
    print(col_name)
    gene_name_list = list(col.index)
    enrichment = gp.enrichr(gene_list = gene_name_list, 
                            description=col_name, 
                            gene_sets='KEGG_2019_Human', 
                            outdir='test/renal')
    print(enrichment.res2d)
    barplot(enrichment.res2d, title=col_name, ofname=col_name)
'''

'for col in sig_cols_down:\n    col_name = col.columns[0]\n    col_name = "_".join((col_name.split(" ")))\n    print(col_name)\n    gene_name_list = list(col.index)\n    enrichment = gp.enrichr(gene_list = gene_name_list, \n                            description=col_name, \n                            gene_sets=\'KEGG_2019_Human\', \n                            outdir=\'test/renal\')\n    print(enrichment.res2d)\n    barplot(enrichment.res2d, title=col_name, ofname=col_name)\n'

In [48]:
def get_dgidb_parameters():
    """
    This function takes no parameters, and will list possible options
    for querying the DGIdb with the dgidb_get_request() function. 
    dgidb_list_parameters() will be called in dgidb_get_request() if 
    incorrect parameter values are given.
    """
        
    print('\nRequired Parameters:')
    print('genes_or_drugs_list (list)')
    print('\nOne of the following:')
    print('genes (bool) OR drugs (bool)\n')
    
    print('Optional Parameters:')
    print('interaction_sources (list (case-sensitive): ["DrugBank","PharmGKB","TALC","TEND","TTD"])')
    print('interaction_types (list (not case-sensitive): ["activator", "inhibitor", "unknown"])')
    print('fda_approved_drug (bool)')
    print('immunotherapy (bool)')
    print('anti_neoplastic (bool)')
    print('clinically_actionable (bool)')
    print('druggable_genome (bool)')
    print('drug_resistance (bool)')
    print('gene_categories (list (not case-sensitive): ["kinase", "dna repair", "tumor suppressor"])')
    print('source_trust_levels (list (not case-sensitive): ["Expert curated", "Non-curated"])')
    
    print('\nFor more information on function parameters and DGIdb, see:')
    print('http://www.dgidb.org/api')
    
    return

def dgidb_get_request(genes_or_drugs_list,
                      genes=True, drugs=False,
                      interaction_sources=[], 
                      interaction_types=[], 
                      fda_approved_drug=False, 
                      immunotherapy=False, 
                      anti_neoplastic=False, 
                      clinically_actionable=False, 
                      druggable_genome=False, 
                      drug_resistance=False, 
                      gene_categories=[], 
                      source_trust_levels=[]):
   
    url = 'http://www.dgidb.org/api/v2/interactions.json?'
    valid_interaction_sources = ["DrugBank","PharmGKB","TALC","TEND","TTD"]
    valid_interaction_types = ["activator", "inhibitor", "unknown"]
    valid_gene_categories = ["kinase", "dna repair", "tumor suppressor"]
    valid_source_trust_levels = ["expert curated", "non-curated"]
    try:
    
        if genes == True:
            url += 'genes='
            if len(genes_or_drugs_list) == 1:
                element = genes_or_drugs_list[0]
                if '(' in element:
                    url += element[:-3]
                else:
                    url += element
            else:
                for i, gene in enumerate(genes_or_drugs_list):
                    if '(' in gene:
                        genes_or_drugs_list[i] = gene[:-3]     

                url += ','.join(genes_or_drugs_list)

        elif drugs == True:
            url += 'drugs='
            url += ','.join(genes_or_drugs_list)
        
        if (drugs == False and genes == False) or (drugs == True and genes == True):
            raise Exception("genes_or_drugs is ambiguous. Please specify if it is a list of genes or drugs.")
        
        unique_sources = list(set(interaction_sources))
        if len(unique_sources) == 1 and unique_sources[0] in valid_interaction_sources:
            url += '&interaction_sources=' + unique_sources[0]
           
        elif len(unique_sources) > 1:
            url += '&interaction_sources='
            for source in unique_sources:
                if source not in valid_interaction_sources:
                    raise Exception("Invalid input interaction_sources: {}".format(source))
                elif source in valid_interaction_sources and source != unique_sources[-1]:
                    url += source + ','
                else:
                    url += source
                    
        unique_types = list(set(interaction_types))
        if len(unique_types) == 1 and unique_types[0] in valid_interaction_types:
            url += '&interaction_types=' + unique_types[0]
        elif len(unique_types) > 1:
            url += '&interaction_types='
            for types in unique_types:
                if types not in valid_interaction_types:
                    raise Exception("Invalid input interaction_types: {}".format(types))
                elif types in valid_interaction_types and types != unique_types[-1]:
                    url += types + ","
                else:
                    url += types

        if fda_approved_drug == True:
            url += '&fda_approved_drug=true'
            
        if immunotherapy == True:
            url += '&immunotherapy=true'
            
        if anti_neoplastic == True:
            url += '&anti_neoplastic=true'
        
        if clinically_actionable == True:
            url += '&clinically_actionable=true'
            
        if druggable_genome == True:
            url += '&druggable_genome=true'
            
        if drug_resistance == True:
            url += '&drug_resistance=true'
        
        unique_categories = list(set(gene_categories))
        if len(unique_categories) == 1:
            url += '&gene_categories=' + unique_categories[0]
            
        elif len(unique_categories) > 1:
            url += '&gene_categories='
            for cat in unique_categories:
                if cat.lower() not in valid_gene_categories:
                    raise Exception("Invalid gene category: {}".format(cat))
                elif cat.lower() == 'dna repair':
                    url += 'dna%20repair'
                elif cat.lower() == 'tumor suppressor':
                    url += 'tumor%20suppressor'
                else:
                    url += cat.lower()
                if cat.lower() in valid_gene_categories and cat.lower() != unique_categories[-1]:
                    url += ","
                    
        unique_levels = list(set(source_trust_levels))            
        if len(unique_levels) == 1:
            if unique_levels[0].lower() not in valid_source_trust_levels:
                raise Exception("Invalid input source_trust_levels: {}".format(unique_levels[0]))
            if unique_levels[0].lower() == "expert curated":
                url += '&source_trust_levels=Expert%20curated'
            else:
                url += '&source_trust_levels=Non-curated'
        
        elif len(unique_levels) > 1:
            url += '&source_trust_levels='
            for item in unique_levels:
                if item.lower() not in valid_source_trust_levels:
                    raise Exception("Invalid source_trust_level: {}".format(item))
                
            url += 'Expert%20curated,Non-curated'
                    
            
        #print("This is the full URL to your GET request:")
        #print(url)
        r = requests.get(url)
        #print("\nSee www.dgidb.org/api or this function's docstring for further explanation and resources on valid parameter inputs")
        
        return r.json()
    
    except Exception as e:
        print("Invalid parameter value: ")
        print(e)
        get_dgidb_parameters()
        
        return

In [49]:
dgidb_requests_up = []
sig_genes_up = []
for i, col in enumerate(sig_cols_up):
    sig_genes_up.append(list(col.index))

In [50]:
dgidb_requests_up = list(map(dgidb_get_request, sig_genes_up))

In [71]:
test = dgidb_requests_up[0]
new_file = 'test.json'

In [72]:
with open(new_file, 'w+') as file:
    json.dump(test, file)
    file.close()

In [73]:
with open(new_file, 'rb') as fin:
    pj = simdjson.ParsedJson(fin.read())
    fin.close()

In [74]:
%%timeit
pj.items('.matchedTerms')

32.3 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [75]:
%%timeit
test.get('matchedTerms')

166 ns ± 5.14 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [76]:
%%timeit
test['matchedTerms']

141 ns ± 24.8 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [98]:
%%timeit
for item in pj.items('.matchedTerms'):
    interactions = item['interactions']
    if len(interactions) > 0:
        print('\nGene:', item['geneName'])
        for item in interactions:
            print('Drug:', item['drugName'])
            print(item['interactionTypes'])
        print('\n')
    else:
        print('Gene:', item['geneName'])

Gene: ACBD3
Gene: AVL9
Gene: COG1
Gene: COG4
Gene: GOLPH3L
Gene: IGF2BP3
Gene: RAB3GAP1

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269
[]
Drug: FLAVIN ADENINE DINUCLEOTIDE
[]
Drug: SPERMIDINE
[]
Drug: FOTEMUSTINE
[]


Gene: ACBD3
Gene: AVL9
Gene: COG1
Gene: COG4
Gene: GOLPH3L
Gene: IGF2BP3
Gene: RAB3GAP1

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269
[]
Drug: FLAVIN ADENINE DINUCLEOTIDE
[]
Drug: SPERMIDINE
[]
Drug: FOTEMUSTINE
[]


Gene: ACBD3
Gene: AVL9
Gene: COG1
Gene: COG4
Gene: GOLPH3L
Gene: IGF2BP3
Gene: RAB3GAP1

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269
[]
Drug: FLAVIN ADENINE DINUCLEOTIDE
[]
Drug: SPERMIDINE
[]
Drug: FOTEMUSTINE
[]


Gene: ACBD3
Gene: AVL9
Gene: COG1
Gene: COG4
Gene: GOLPH3L
Gene: IGF2BP3
Gene: RAB3GAP1

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269

In [100]:
#pj.items('.matchedTerms')

In [54]:
'''dgidb_requests_down = []

for col in sig_cols_down:
    sig_genes = list(col.index)
    json_obj = bf.dgidb_get_request(sig_genes)
    dgidb_requests_down.append(json_obj)
    print('\n')
'''

"dgidb_requests_down = []\n\nfor col in sig_cols_down:\n    sig_genes = list(col.index)\n    json_obj = bf.dgidb_get_request(sig_genes)\n    dgidb_requests_down.append(json_obj)\n    print('\n')\n"

In [102]:
#%%timeit
def dgidb_json_parse(json_obj):
    json_obj = json_obj['matchedTerms']
    drugs = {}
    for item in json_obj:
        if len(item['interactions']) > 0:
            interactions_list = []
            #interactions_dict = {}
            for interaction in item['interactions']:
                interactions_list.append(interaction['drugName'])
                #print('Gene:', interaction['drugName'], '\nInteraction Type:', interaction['interaction_types'])
                #interactions_dict[interaction['drugName']] = interaction['interaction_types']
            drugs[item['geneName']] = interactions_list
    for k, v in drugs.items():
        print('Gene: ' + k)
        print('Drugs: ')
        print(v)
        print('\n')
            
    return drugs

def json_parse2(json_obj, matchedTerms):
    new_json = {}
    new_list = []
    for i in matchedTerms:
        print('Gene:', i.get('geneName'))
        if len(i.get('interactions')) == 0:
            continue
        else:
            for item in i.get('interactions'):
                print('Drug Name:', item.get('drugName'))
                print('Interaction Type:', item.get('interaction_types'))
        print('\n')
        
def json_parse3(json_obj_matched_terms):
    drugs = {}
    for item in json_obj_matched_terms:
        interaction_list = item['interactions']
        gene_name = item['geneName']
        print('Gene:', gene_name)
        if len(interaction_list) > 0:
            #drugs[item['geneName']] = map(interaction_list.get('drugName'), interaction_list)
            drugs[gene_name] = interaction_list
            print('Drugs', interaction_list)
            
    return drugs

def json_parse4(json_obj_matched_terms):
    for item in json_obj_matched_terms:
        interactions = item['interactions']
        if len(interactions) > 0:
            print('\nGene:', item['geneName'])
            for item in interactions:
                print('Drug:', item['drugName'])
                print(item['interactionTypes'])
            print('\n')
        else:
            print('Gene:', item['geneName'])

In [104]:
%%timeit
print('UP-REGULATED:\n')
for i, request in enumerate(dgidb_requests_up):
    print('Genes and drugs for: ' + sig_cols_up[i].columns[0] +':\n')
    drug_dict_up = dgidb_json_parse(request)

UP-REGULATED:

Genes and drugs for: tumor_site_Other_P_values:

Gene: TXNRD1
Drugs: 
['ARSENIC TRIOXIDE', 'MOTEXAFIN GADOLINIUM', 'CHEMBL449269', 'FLAVIN ADENINE DINUCLEOTIDE', 'SPERMIDINE', 'FOTEMUSTINE']


Genes and drugs for: tumor_size_in_cm_Above_Mean(6.51)_P_values:

Gene: TXNRD1
Drugs: 
['ARSENIC TRIOXIDE', 'MOTEXAFIN GADOLINIUM', 'CHEMBL449269', 'FLAVIN ADENINE DINUCLEOTIDE', 'SPERMIDINE', 'FOTEMUSTINE']


Genes and drugs for: tumor_focality_Multifocal_P_values:

Gene: HDAC4
Drugs: 
['TASQUINIMOD', 'BELINOSTAT', 'CUDC-101', 'DACINOSTAT', 'CHEMBL1213492', 'PANOBINOSTAT', 'QUISINOSTAT', 'ROMIDEPSIN', 'SCRIPTAID', 'CHEMBL3110004', 'TRICHOSTATIN', 'CHEMBL1851943', 'PANOBINOSTAT LACTATE', 'ENTINOSTAT', 'CHEMBL100014', 'VALPROIC ACID', 'SODIUM PHENYLBUTYRATE', 'ABEXINOSTAT', 'VORINOSTAT', 'CUDC-907', 'TACEDINALINE', 'MOCETINOSTAT', 'CHEMBL191482']


Gene: KRT8
Drugs: 
['TENECTEPLASE']


Gene: PPIA
Drugs: 
['L-PROLINE', 'CYCLOSPORINE', 'VOCLOSPORIN', 'CHEMBL1269597']


Genes and drugs

['HEXANE', 'GW620972X', 'CHEMBL210618', 'RGB-286638', 'SP-600125', 'CC-401', 'TANZISERTIB', 'Bentamapimod', 'CAMPTOTHECIN']


Gene: NPEPL1
Drugs: 
['TOSEDOSTAT']


Gene: P2RX4
Drugs: 
['ESLICARBAZEPINE ACETATE']


Gene: PMVK
Drugs: 
['DIPHOSPHOMELAVONATE']


Gene: PSMD8
Drugs: 
['CARFILZOMIB', 'IXAZOMIB CITRATE', 'BORTEZOMIB', 'OPROZOMIB']


Genes and drugs for: anesthesia_steroids_Yes_P_values:

Gene: CD38
Drugs: 
['DARATUMUMAB', 'ISATUXIMAB', 'SAR-650984', 'HuMax-CD38', 'THROMBIN']


Gene: HSD17B11
Drugs: 
['ETIOCHOLANOLONE']


Gene: MPI
Drugs: 
['SULFOXONE', 'SULFACETAMIDE']


Genes and drugs for: tumor_status_at_12months_follow_up_With Tumor_P_values:

Gene: ASPH
Drugs: 
['ASPARTIC ACID', 'SUCCINIC ACID']


Gene: CALR
Drugs: 
['TRETINOIN', 'GENTAMICIN', 'TENECTEPLASE', 'ETRETINATE', 'MELATONIN', 'OLTIPRAZ']


Gene: GLS
Drugs: 
['6-DIAZO-5-OXO-L-NORLEUCINE', 'METHOTREXATE', 'GLUTAMINE', 'DEXAMETHASONE', 'L-GLUTAMATE']


Gene: IRS2
Drugs: 
['ASPIRIN', 'DEXAMETHASONE']


Gene: PYCR2
D

Gene: HSP90AB1
Drugs: 
['RETASPIMYCIN', 'NVP-AUY922', 'GANETESPIB', 'TANESPIMYCIN', 'RETASPIMYCIN HYDROCHLORIDE (CHEMBL377559)', 'ONALESPIB', 'CHEMBL399530', 'ALVESPIMYCIN', 'RADICICOL', 'GELDANAMYCIN', 'CHEMBL383189', 'CHEMBL2204502', 'CHEMBL553939', 'CHEMBL113690', 'CHEMBL2170470']


Gene: TXNRD1
Drugs: 
['ARSENIC TRIOXIDE', 'MOTEXAFIN GADOLINIUM', 'CHEMBL449269', 'FLAVIN ADENINE DINUCLEOTIDE', 'SPERMIDINE', 'FOTEMUSTINE']


Genes and drugs for: margin_status_Margin(s) involved by invasive carcinoma_P_values:

Gene: ATP2A2
Drugs: 
['ENOXACIN', 'ISTAROXIME', 'CELECOXIB']


Gene: B3GAT3
Drugs: 
['CHEMBL1162071', 'URIDINE_DIPHOSPHATE']


Gene: CALR
Drugs: 
['TRETINOIN', 'GENTAMICIN', 'TENECTEPLASE', 'ETRETINATE', 'MELATONIN', 'OLTIPRAZ']


Gene: CD38
Drugs: 
['DARATUMUMAB', 'ISATUXIMAB', 'SAR-650984', 'HuMax-CD38', 'THROMBIN']


Gene: ECE1
Drugs: 
['CHEMBL571153', 'DAGLUTRIL', 'INDOLE-3-BUTYRIC ACID', 'ARDENERMIN']


Gene: G6PD
Drugs: 
['GLYCOLIC ACID', 'SODIUM SULFATE', 'SULFAMETHOXAZO

['CHEMBL23552', 'FORMIC ACID']


Gene: EPB41
Drugs: 
['BUPROPION']


Gene: FGA
Drugs: 
['Sucralfate', 'FIBRINOLYSIN, HUMAN', 'EPTIFIBATIDE', 'ANISTREPLASE', 'ANCROD', 'ABCIXIMAB', 'RETEPLASE', 'ALTEPLASE', 'ALFIMEPRASE', 'TENECTEPLASE', 'TIROFIBAN']


Gene: FGB
Drugs: 
['Sucralfate', 'FIBRINOLYSIN, HUMAN', 'ALFIMEPRASE', 'EPTIFIBATIDE', 'TIROFIBAN', 'ABCIXIMAB']


Gene: FGG
Drugs: 
['Sucralfate', 'FIBRINOLYSIN, HUMAN', 'EPTIFIBATIDE', 'ABCIXIMAB', 'ALFIMEPRASE', 'TIROFIBAN']


Gene: GNAS
Drugs: 
['COLFORSIN', 'FLUOROURACIL']


Gene: HBA2
Drugs: 
['EFAPROXIRAL']


Gene: HBB
Drugs: 
['IRON DEXTRAN', 'EFAPROXIRAL', 'CHEMBL1205452', 'CHEMBL77562', 'SEBACIC ACID', 'CHEMBL76029']


Gene: IL6ST
Drugs: 
['SA237']


Gene: MAPK14
Drugs: 
['PF-03715455', 'CHEMBL1236539', 'Acumapimod', 'AZD-6703', 'BMS-582949', 'PROPOXYPHENE', 'DILMAPIMOD', 'DORAMAPIMOD', 'LOSMAPIMOD', 'CHEMBL1951415', 'PH-797804', 'CHEMBL306380', 'RALIMETINIB', 'SB-202190', 'SB-203580', 'SB-220025', 'CHEMBL313417', 'TAK-715', 'TA

Drugs: 
['ARSENIC TRIOXIDE', 'MOTEXAFIN GADOLINIUM', 'CHEMBL449269', 'FLAVIN ADENINE DINUCLEOTIDE', 'SPERMIDINE', 'FOTEMUSTINE']


Genes and drugs for: tumor_size_in_cm_Above_Mean(6.51)_P_values:

Gene: TXNRD1
Drugs: 
['ARSENIC TRIOXIDE', 'MOTEXAFIN GADOLINIUM', 'CHEMBL449269', 'FLAVIN ADENINE DINUCLEOTIDE', 'SPERMIDINE', 'FOTEMUSTINE']


Genes and drugs for: tumor_focality_Multifocal_P_values:

Gene: HDAC4
Drugs: 
['TASQUINIMOD', 'BELINOSTAT', 'CUDC-101', 'DACINOSTAT', 'CHEMBL1213492', 'PANOBINOSTAT', 'QUISINOSTAT', 'ROMIDEPSIN', 'SCRIPTAID', 'CHEMBL3110004', 'TRICHOSTATIN', 'CHEMBL1851943', 'PANOBINOSTAT LACTATE', 'ENTINOSTAT', 'CHEMBL100014', 'VALPROIC ACID', 'SODIUM PHENYLBUTYRATE', 'ABEXINOSTAT', 'VORINOSTAT', 'CUDC-907', 'TACEDINALINE', 'MOCETINOSTAT', 'CHEMBL191482']


Gene: KRT8
Drugs: 
['TENECTEPLASE']


Gene: PPIA
Drugs: 
['L-PROLINE', 'CYCLOSPORINE', 'VOCLOSPORIN', 'CHEMBL1269597']


Genes and drugs for: histologic_grade_G3or4_P_values:

Gene: IRS2
Drugs: 
['ASPIRIN', 'DEXAM

...

['RETASPIMYCIN', 'NVP-AUY922', 'GANETESPIB', 'TANESPIMYCIN', 'RETASPIMYCIN HYDROCHLORIDE (CHEMBL377559)', 'ONALESPIB', 'CHEMBL399530', 'ALVESPIMYCIN', 'RADICICOL', 'GELDANAMYCIN', 'CHEMBL383189', 'CHEMBL2204502', 'CHEMBL553939', 'CHEMBL113690', 'CHEMBL2170470']


Gene: TXNRD1
Drugs: 
['ARSENIC TRIOXIDE', 'MOTEXAFIN GADOLINIUM', 'CHEMBL449269', 'FLAVIN ADENINE DINUCLEOTIDE', 'SPERMIDINE', 'FOTEMUSTINE']


Genes and drugs for: margin_status_Margin(s) involved by invasive carcinoma_P_values:

Gene: ATP2A2
Drugs: 
['ENOXACIN', 'ISTAROXIME', 'CELECOXIB']


Gene: B3GAT3
Drugs: 
['CHEMBL1162071', 'URIDINE_DIPHOSPHATE']


Gene: CALR
Drugs: 
['TRETINOIN', 'GENTAMICIN', 'TENECTEPLASE', 'ETRETINATE', 'MELATONIN', 'OLTIPRAZ']


Gene: CD38
Drugs: 
['DARATUMUMAB', 'ISATUXIMAB', 'SAR-650984', 'HuMax-CD38', 'THROMBIN']


Gene: ECE1
Drugs: 
['CHEMBL571153', 'DAGLUTRIL', 'INDOLE-3-BUTYRIC ACID', 'ARDENERMIN']


Gene: G6PD
Drugs: 
['GLYCOLIC ACID', 'SODIUM SULFATE', 'SULFAMETHOXAZOLE', 'NITROFURANTOIN', 

Gene: EPB41
Drugs: 
['BUPROPION']


Gene: FGA
Drugs: 
['Sucralfate', 'FIBRINOLYSIN, HUMAN', 'EPTIFIBATIDE', 'ANISTREPLASE', 'ANCROD', 'ABCIXIMAB', 'RETEPLASE', 'ALTEPLASE', 'ALFIMEPRASE', 'TENECTEPLASE', 'TIROFIBAN']


Gene: FGB
Drugs: 
['Sucralfate', 'FIBRINOLYSIN, HUMAN', 'ALFIMEPRASE', 'EPTIFIBATIDE', 'TIROFIBAN', 'ABCIXIMAB']


Gene: FGG
Drugs: 
['Sucralfate', 'FIBRINOLYSIN, HUMAN', 'EPTIFIBATIDE', 'ABCIXIMAB', 'ALFIMEPRASE', 'TIROFIBAN']


Gene: GNAS
Drugs: 
['COLFORSIN', 'FLUOROURACIL']


Gene: HBA2
Drugs: 
['EFAPROXIRAL']


Gene: HBB
Drugs: 
['IRON DEXTRAN', 'EFAPROXIRAL', 'CHEMBL1205452', 'CHEMBL77562', 'SEBACIC ACID', 'CHEMBL76029']


Gene: IL6ST
Drugs: 
['SA237']


Gene: MAPK14
Drugs: 
['PF-03715455', 'CHEMBL1236539', 'Acumapimod', 'AZD-6703', 'BMS-582949', 'PROPOXYPHENE', 'DILMAPIMOD', 'DORAMAPIMOD', 'LOSMAPIMOD', 'CHEMBL1951415', 'PH-797804', 'CHEMBL306380', 'RALIMETINIB', 'SB-202190', 'SB-203580', 'SB-220025', 'CHEMBL313417', 'TAK-715', 'TALMAPIMOD', 'VX-702', 'VX-745', 'G

['ARSENIC TRIOXIDE', 'MOTEXAFIN GADOLINIUM', 'CHEMBL449269', 'FLAVIN ADENINE DINUCLEOTIDE', 'SPERMIDINE', 'FOTEMUSTINE']


Genes and drugs for: tumor_size_in_cm_Above_Mean(6.51)_P_values:

Gene: TXNRD1
Drugs: 
['ARSENIC TRIOXIDE', 'MOTEXAFIN GADOLINIUM', 'CHEMBL449269', 'FLAVIN ADENINE DINUCLEOTIDE', 'SPERMIDINE', 'FOTEMUSTINE']


Genes and drugs for: tumor_focality_Multifocal_P_values:

Gene: HDAC4
Drugs: 
['TASQUINIMOD', 'BELINOSTAT', 'CUDC-101', 'DACINOSTAT', 'CHEMBL1213492', 'PANOBINOSTAT', 'QUISINOSTAT', 'ROMIDEPSIN', 'SCRIPTAID', 'CHEMBL3110004', 'TRICHOSTATIN', 'CHEMBL1851943', 'PANOBINOSTAT LACTATE', 'ENTINOSTAT', 'CHEMBL100014', 'VALPROIC ACID', 'SODIUM PHENYLBUTYRATE', 'ABEXINOSTAT', 'VORINOSTAT', 'CUDC-907', 'TACEDINALINE', 'MOCETINOSTAT', 'CHEMBL191482']


Gene: KRT8
Drugs: 
['TENECTEPLASE']


Gene: PPIA
Drugs: 
['L-PROLINE', 'CYCLOSPORINE', 'VOCLOSPORIN', 'CHEMBL1269597']


Genes and drugs for: histologic_grade_G3or4_P_values:

Gene: IRS2
Drugs: 
['ASPIRIN', 'DEXAMETHASONE

Gene: P2RX4
Drugs: 
['ESLICARBAZEPINE ACETATE']


Gene: PMVK
Drugs: 
['DIPHOSPHOMELAVONATE']


Gene: PSMD8
Drugs: 
['CARFILZOMIB', 'IXAZOMIB CITRATE', 'BORTEZOMIB', 'OPROZOMIB']


Genes and drugs for: anesthesia_steroids_Yes_P_values:

Gene: CD38
Drugs: 
['DARATUMUMAB', 'ISATUXIMAB', 'SAR-650984', 'HuMax-CD38', 'THROMBIN']


Gene: HSD17B11
Drugs: 
['ETIOCHOLANOLONE']


Gene: MPI
Drugs: 
['SULFOXONE', 'SULFACETAMIDE']


Genes and drugs for: tumor_status_at_12months_follow_up_With Tumor_P_values:

Gene: ASPH
Drugs: 
['ASPARTIC ACID', 'SUCCINIC ACID']


Gene: CALR
Drugs: 
['TRETINOIN', 'GENTAMICIN', 'TENECTEPLASE', 'ETRETINATE', 'MELATONIN', 'OLTIPRAZ']


Gene: GLS
Drugs: 
['6-DIAZO-5-OXO-L-NORLEUCINE', 'METHOTREXATE', 'GLUTAMINE', 'DEXAMETHASONE', 'L-GLUTAMATE']


Gene: IRS2
Drugs: 
['ASPIRIN', 'DEXAMETHASONE']


Gene: PYCR2
Drugs: 
['CHEMBL1161866', 'L-PROLINE']


Gene: TARS
Drugs: 
['THREONINE']


Genes and drugs for: tumor_status_at_24months_follow_up_With Tumor_P_values:

Gene: MARK2

['RETASPIMYCIN', 'NVP-AUY922', 'GANETESPIB', 'TANESPIMYCIN', 'RETASPIMYCIN HYDROCHLORIDE (CHEMBL377559)', 'ONALESPIB', 'CHEMBL399530', 'ALVESPIMYCIN', 'RADICICOL', 'GELDANAMYCIN', 'CHEMBL383189', 'CHEMBL2204502', 'CHEMBL553939', 'CHEMBL113690', 'CHEMBL2170470']


Gene: TXNRD1
Drugs: 
['ARSENIC TRIOXIDE', 'MOTEXAFIN GADOLINIUM', 'CHEMBL449269', 'FLAVIN ADENINE DINUCLEOTIDE', 'SPERMIDINE', 'FOTEMUSTINE']


Genes and drugs for: margin_status_Margin(s) involved by invasive carcinoma_P_values:

Gene: ATP2A2
Drugs: 
['ENOXACIN', 'ISTAROXIME', 'CELECOXIB']


Gene: B3GAT3
Drugs: 
['CHEMBL1162071', 'URIDINE_DIPHOSPHATE']


Gene: CALR
Drugs: 
['TRETINOIN', 'GENTAMICIN', 'TENECTEPLASE', 'ETRETINATE', 'MELATONIN', 'OLTIPRAZ']


Gene: CD38
Drugs: 
['DARATUMUMAB', 'ISATUXIMAB', 'SAR-650984', 'HuMax-CD38', 'THROMBIN']


Gene: ECE1
Drugs: 
['CHEMBL571153', 'DAGLUTRIL', 'INDOLE-3-BUTYRIC ACID', 'ARDENERMIN']


Gene: G6PD
Drugs: 
['GLYCOLIC ACID', 'SODIUM SULFATE', 'SULFAMETHOXAZOLE', 'NITROFURANTOIN', 

In [103]:
%%timeit
for i, request in enumerate(dgidb_requests_up):
    print('Genes and drugs for: ' + sig_cols_up[i].columns[0] +':\n')
    request = dgidb_requests_up[i]
    drug_dict_up = json_parse2(request, request['matchedTerms'])


Genes and drugs for: tumor_site_Other_P_values:

Gene: ACBD3
Gene: AVL9
Gene: COG1
Gene: COG4
Gene: GOLPH3L
Gene: IGF2BP3
Gene: RAB3GAP1
Gene: TXNRD1
Drug Name: ARSENIC TRIOXIDE
Interaction Type: None
Drug Name: MOTEXAFIN GADOLINIUM
Interaction Type: None
Drug Name: CHEMBL449269
Interaction Type: None
Drug Name: FLAVIN ADENINE DINUCLEOTIDE
Interaction Type: None
Drug Name: SPERMIDINE
Interaction Type: None
Drug Name: FOTEMUSTINE
Interaction Type: None


Genes and drugs for: tumor_size_in_cm_Above_Mean(6.51)_P_values:

Gene: ACBD3
Gene: GOLPH3L
Gene: GSDME
Gene: IGF2BP3
Gene: TXNRD1
Drug Name: ARSENIC TRIOXIDE
Interaction Type: None
Drug Name: MOTEXAFIN GADOLINIUM
Interaction Type: None
Drug Name: CHEMBL449269
Interaction Type: None
Drug Name: FLAVIN ADENINE DINUCLEOTIDE
Interaction Type: None
Drug Name: SPERMIDINE
Interaction Type: None
Drug Name: FOTEMUSTINE
Interaction Type: None


Genes and drugs for: tumor_focality_Multifocal_P_values:

Gene: AGL
Gene: ATG5
Gene: CTPS2
Gene: DCAF7


Drug Name: FLUOROURACIL
Interaction Type: None


Gene: GYPC
Gene: HBA2
Drug Name: EFAPROXIRAL
Interaction Type: None


Gene: HBB
Drug Name: IRON DEXTRAN
Interaction Type: None
Drug Name: EFAPROXIRAL
Interaction Type: None
Drug Name: CHEMBL1205452
Interaction Type: None
Drug Name: CHEMBL77562
Interaction Type: None
Drug Name: SEBACIC ACID
Interaction Type: None
Drug Name: CHEMBL76029
Interaction Type: None


Gene: HBE1
Gene: HMBS
Gene: HRG
Gene: IL6ST
Drug Name: SA237
Interaction Type: None


Gene: INTU
Gene: KCMF1
Gene: LARP6
Gene: LSM12
Gene: MAPK14
Drug Name: PF-03715455
Interaction Type: None
Drug Name: CHEMBL1236539
Interaction Type: None
Drug Name: Acumapimod
Interaction Type: None
Drug Name: AZD-6703
Interaction Type: None
Drug Name: BMS-582949
Interaction Type: None
Drug Name: PROPOXYPHENE
Interaction Type: None
Drug Name: DILMAPIMOD
Interaction Type: None
Drug Name: DORAMAPIMOD
Interaction Type: None
Drug Name: LOSMAPIMOD
Interaction Type: None
Drug Name: CHEMBL1951415
Interact

Drug Name: DARATUMUMAB
Interaction Type: None
Drug Name: ISATUXIMAB
Interaction Type: None
Drug Name: SAR-650984
Interaction Type: None
Drug Name: HuMax-CD38
Interaction Type: None
Drug Name: THROMBIN
Interaction Type: None


Gene: COPA
Gene: CRELD2
Gene: DHCR24
Gene: DNALI1
Gene: ECE1
Drug Name: CHEMBL571153
Interaction Type: None
Drug Name: DAGLUTRIL
Interaction Type: None
Drug Name: INDOLE-3-BUTYRIC ACID
Interaction Type: None
Drug Name: ARDENERMIN
Interaction Type: None


Gene: EIF2B5
Gene: EIF3E
Gene: ERLEC1
Gene: FAM192A
Gene: FXYD1
Gene: G6PD
Drug Name: GLYCOLIC ACID
Interaction Type: None
Drug Name: SODIUM SULFATE
Interaction Type: None
Drug Name: SULFAMETHOXAZOLE
Interaction Type: None
Drug Name: NITROFURANTOIN
Interaction Type: None
Drug Name: TRAMETINIB
Interaction Type: None
Drug Name: TRIAMCINOLONE
Interaction Type: None
Drug Name: QUININE SULFATE
Interaction Type: None
Drug Name: CO-TRIMOXAZOLE
Interaction Type: None
Drug Name: SULFISOXAZOLE
Interaction Type: None
Drug Na

Gene: C12ORF29
Gene: CALR
Drug Name: TRETINOIN
Interaction Type: None
Drug Name: GENTAMICIN
Interaction Type: None
Drug Name: TENECTEPLASE
Interaction Type: None
Drug Name: ETRETINATE
Interaction Type: None
Drug Name: MELATONIN
Interaction Type: None
Drug Name: OLTIPRAZ
Interaction Type: None


Gene: CD44
Drug Name: HYALURONIC ACID
Interaction Type: None
Drug Name: STAUROSPORINE
Interaction Type: None
Drug Name: INTERFERON GAMA-1B
Interaction Type: None
Drug Name: MYCOPHENOLIC ACID
Interaction Type: None
Drug Name: GENTAMICIN
Interaction Type: None
Drug Name: MOMETASONE FUROATE
Interaction Type: None


Gene: DST
Gene: GALNT2
Gene: GLS
Drug Name: 6-DIAZO-5-OXO-L-NORLEUCINE
Interaction Type: None
Drug Name: METHOTREXATE
Interaction Type: None
Drug Name: GLUTAMINE
Interaction Type: None
Drug Name: DEXAMETHASONE
Interaction Type: None
Drug Name: L-GLUTAMATE
Interaction Type: None


Gene: GORASP2
Gene: HSP90AB1
Drug Name: RETASPIMYCIN
Interaction Type: None
Drug Name: NVP-AUY922
Interaction

Gene: BCCIP
Gene: BPGM
Gene: C4B
Drug Name: GLOBULIN, IMMUNE
Interaction Type: None


Gene: C9ORF40
Gene: CA1
Drug Name: METHYCLOTHIAZIDE
Interaction Type: None
Drug Name: ETHOXZOLAMIDE
Interaction Type: None
Drug Name: METHOCARBAMOL
Interaction Type: None
Drug Name: BENDROFLUMETHIAZIDE
Interaction Type: None
Drug Name: BENZTHIAZIDE
Interaction Type: None
Drug Name: CYCLOTHIAZIDE
Interaction Type: None
Drug Name: METHAZOLAMIDE
Interaction Type: None
Drug Name: HYDROFLUMETHIAZIDE
Interaction Type: None
Drug Name: ZONISAMIDE
Interaction Type: None
Drug Name: HYDROCHLOROTHIAZIDE
Interaction Type: None
Drug Name: TRICHLORMETHIAZIDE
Interaction Type: None
Drug Name: ETHINAMATE
Interaction Type: None
Drug Name: DIAZOXIDE
Interaction Type: None
Drug Name: DICHLORPHENAMIDE
Interaction Type: None
Drug Name: ACETAZOLAMIDE
Interaction Type: None
Drug Name: BENZOLAMIDE
Interaction Type: None
Drug Name: BRINZOLAMIDE
Interaction Type: None
Drug Name: CHLORTHALIDONE
Interaction Type: None
Drug Name: 

...

Gene: SRSF9
Gene: TANC2
Gene: TARS
Drug Name: THREONINE
Interaction Type: None


Gene: UGGT1
Genes and drugs for: tobacco_smoking_history_Current Smoker_P_values:

Gene: AEBP1
Gene: BARD1
Gene: CPN1
Drug Name: PYRAZINAMIDE
Interaction Type: None


Gene: IGSF3
Gene: MAP4K4
Gene: SHBG
Drug Name: SPIRONOLACTONE
Interaction Type: None
Drug Name: BUSULFAN
Interaction Type: None
Drug Name: LISINOPRIL
Interaction Type: None
Drug Name: NORGESTREL
Interaction Type: None
Drug Name: DROLOXIFENE
Interaction Type: None


Gene: SPECC1L
Genes and drugs for: anesthesia_insulin_Yes_P_values:

Gene: ARHGEF16
Gene: ATG7
Gene: ATP13A3
Gene: C6ORF106
Gene: CAMK1
Gene: CD44
Drug Name: HYALURONIC ACID
Interaction Type: None
Drug Name: STAUROSPORINE
Interaction Type: None
Drug Name: INTERFERON GAMA-1B
Interaction Type: None
Drug Name: MYCOPHENOLIC ACID
Interaction Type: None
Drug Name: GENTAMICIN
Interaction Type: None
Drug Name: MOMETASONE FUROATE
Interaction Type: None


Gene: CLIC6
Gene: CSE1L
Gene: GALNT2


Gene: ACBD3
Gene: GOLPH3L
Gene: GSDME
Gene: IGF2BP3
Gene: TXNRD1
Drug Name: ARSENIC TRIOXIDE
Interaction Type: None
Drug Name: MOTEXAFIN GADOLINIUM
Interaction Type: None
Drug Name: CHEMBL449269
Interaction Type: None
Drug Name: FLAVIN ADENINE DINUCLEOTIDE
Interaction Type: None
Drug Name: SPERMIDINE
Interaction Type: None
Drug Name: FOTEMUSTINE
Interaction Type: None


Genes and drugs for: tumor_focality_Multifocal_P_values:

Gene: AGL
Gene: ATG5
Gene: CTPS2
Gene: DCAF7
Gene: HDAC4
Drug Name: TASQUINIMOD
Interaction Type: None
Drug Name: BELINOSTAT
Interaction Type: None
Drug Name: CUDC-101
Interaction Type: None
Drug Name: DACINOSTAT
Interaction Type: None
Drug Name: CHEMBL1213492
Interaction Type: None
Drug Name: PANOBINOSTAT
Interaction Type: None
Drug Name: QUISINOSTAT
Interaction Type: None
Drug Name: ROMIDEPSIN
Interaction Type: None
Drug Name: SCRIPTAID
Interaction Type: None
Drug Name: CHEMBL3110004
Interaction Type: None
Drug Name: TRICHOSTATIN
Interaction Type: None
Drug Na

Gene: LSM12
Gene: MAPK14
Drug Name: PF-03715455
Interaction Type: None
Drug Name: CHEMBL1236539
Interaction Type: None
Drug Name: Acumapimod
Interaction Type: None
Drug Name: AZD-6703
Interaction Type: None
Drug Name: BMS-582949
Interaction Type: None
Drug Name: PROPOXYPHENE
Interaction Type: None
Drug Name: DILMAPIMOD
Interaction Type: None
Drug Name: DORAMAPIMOD
Interaction Type: None
Drug Name: LOSMAPIMOD
Interaction Type: None
Drug Name: CHEMBL1951415
Interaction Type: None
Drug Name: PH-797804
Interaction Type: None
Drug Name: CHEMBL306380
Interaction Type: None
Drug Name: RALIMETINIB
Interaction Type: None
Drug Name: SB-202190
Interaction Type: None
Drug Name: SB-203580
Interaction Type: None
Drug Name: SB-220025
Interaction Type: None
Drug Name: CHEMBL313417
Interaction Type: None
Drug Name: TAK-715
Interaction Type: None
Drug Name: TALMAPIMOD
Interaction Type: None
Drug Name: VX-702
Interaction Type: None
Drug Name: VX-745
Interaction Type: None
Drug Name: GSK-610677
Interactio

Gene: FXYD1
Gene: G6PD
Drug Name: GLYCOLIC ACID
Interaction Type: None
Drug Name: SODIUM SULFATE
Interaction Type: None
Drug Name: SULFAMETHOXAZOLE
Interaction Type: None
Drug Name: NITROFURANTOIN
Interaction Type: None
Drug Name: TRAMETINIB
Interaction Type: None
Drug Name: TRIAMCINOLONE
Interaction Type: None
Drug Name: QUININE SULFATE
Interaction Type: None
Drug Name: CO-TRIMOXAZOLE
Interaction Type: None
Drug Name: SULFISOXAZOLE
Interaction Type: None
Drug Name: DIAZEPAM
Interaction Type: None
Drug Name: PREDNISOLONE
Interaction Type: None
Drug Name: SULFASALAZINE
Interaction Type: None
Drug Name: PRIMAQUINE
Interaction Type: None
Drug Name: SODIUM ASCORBATE
Interaction Type: None
Drug Name: ASCORBATE
Interaction Type: None
Drug Name: LIDOCAINE
Interaction Type: None
Drug Name: METOCLOPRAMIDE
Interaction Type: None
Drug Name: PYRIMETHAMINE
Interaction Type: None
Drug Name: PEGLOTICASE
Interaction Type: None
Drug Name: SULFADOXINE
Interaction Type: None
Drug Name: CARMUSTINE
Interac

Interaction Type: None
Drug Name: MOMETASONE FUROATE
Interaction Type: None


Gene: DST
Gene: GALNT2
Gene: GLS
Drug Name: 6-DIAZO-5-OXO-L-NORLEUCINE
Interaction Type: None
Drug Name: METHOTREXATE
Interaction Type: None
Drug Name: GLUTAMINE
Interaction Type: None
Drug Name: DEXAMETHASONE
Interaction Type: None
Drug Name: L-GLUTAMATE
Interaction Type: None


Gene: GORASP2
Gene: HSP90AB1
Drug Name: RETASPIMYCIN
Interaction Type: None
Drug Name: NVP-AUY922
Interaction Type: None
Drug Name: GANETESPIB
Interaction Type: None
Drug Name: TANESPIMYCIN
Interaction Type: None
Drug Name: RETASPIMYCIN HYDROCHLORIDE (CHEMBL377559)
Interaction Type: None
Drug Name: ONALESPIB
Interaction Type: None
Drug Name: CHEMBL399530
Interaction Type: None
Drug Name: ALVESPIMYCIN
Interaction Type: None
Drug Name: RADICICOL
Interaction Type: None
Drug Name: GELDANAMYCIN
Interaction Type: None
Drug Name: CHEMBL383189
Interaction Type: None
Drug Name: CHEMBL2204502
Interaction Type: None
Drug Name: CHEMBL553939
Inte

In [105]:
%%timeit
print('UP-REGULATED:\n')
for i, request in enumerate(dgidb_requests_up):
    #print('Genes and drugs for: ' + sig_cols_up[i].columns[0] +':\n')
    drug_dict_up = json_parse3(request['matchedTerms'])


UP-REGULATED:

Gene: ACBD3
Gene: AVL9
Gene: COG1
Gene: COG4
Gene: GOLPH3L
Gene: IGF2BP3
Gene: RAB3GAP1
Gene: TXNRD1
Drugs [{'interactionId': '18f27be8-5b8a-4164-b1c2-c5d159a022b5', 'interactionTypes': ['inhibitor'], 'drugName': 'ARSENIC TRIOXIDE', 'drugChemblId': 'CHEMBL1200978', 'sources': ['TdgClinicalTrial', 'ChemblInteractions', 'TEND', 'DrugBank'], 'pmids': [17640917], 'score': 5}, {'interactionId': 'fc2b2336-7492-4bd6-bc70-0d9a77feb393', 'interactionTypes': [], 'drugName': 'MOTEXAFIN GADOLINIUM', 'drugChemblId': 'CHEMBL3544910', 'sources': ['TdgClinicalTrial', 'DrugBank'], 'pmids': [], 'score': 2}, {'interactionId': 'dbb5c849-7d78-4b41-a794-74f28c1dbecf', 'interactionTypes': [], 'drugName': 'CHEMBL449269', 'drugChemblId': 'CHEMBL449269', 'sources': ['DrugBank'], 'pmids': [10592235], 'score': 2}, {'interactionId': '7436b218-7b03-44e4-aa5f-2370a8ca904f', 'interactionTypes': [], 'drugName': 'FLAVIN ADENINE DINUCLEOTIDE', 'drugChemblId': 'CHEMBL1232653', 'sources': ['DrugBank'], 'pmi

Drugs [{'interactionId': 'fce537fe-ab0c-441c-93b6-6bd35a06f795', 'interactionTypes': ['allosteric modulator'], 'drugName': 'TASQUINIMOD', 'drugChemblId': 'CHEMBL2107784', 'sources': ['GuideToPharmacologyInteractions'], 'pmids': [], 'score': 1}, {'interactionId': 'a8d025f9-cd1b-4adc-af35-bfdd9cec32fa', 'interactionTypes': ['inhibitor'], 'drugName': 'BELINOSTAT', 'drugChemblId': 'CHEMBL408513', 'sources': ['TALC', 'MyCancerGenome', 'TdgClinicalTrial', 'GuideToPharmacologyInteractions', 'ChemblInteractions'], 'pmids': [], 'score': 5}, {'interactionId': '15040527-e093-48c7-8588-378ea387f4c3', 'interactionTypes': ['inhibitor'], 'drugName': 'CUDC-101', 'drugChemblId': 'CHEMBL598797', 'sources': ['TALC', 'TdgClinicalTrial', 'GuideToPharmacologyInteractions', 'ChemblInteractions'], 'pmids': [], 'score': 4}, {'interactionId': 'f58271af-5984-4f40-a86f-fb3807c3089a', 'interactionTypes': ['inhibitor'], 'drugName': 'DACINOSTAT', 'drugChemblId': 'CHEMBL356066', 'sources': ['GuideToPharmacologyIntera

Drugs [{'interactionId': '18f27be8-5b8a-4164-b1c2-c5d159a022b5', 'interactionTypes': ['inhibitor'], 'drugName': 'ARSENIC TRIOXIDE', 'drugChemblId': 'CHEMBL1200978', 'sources': ['TdgClinicalTrial', 'ChemblInteractions', 'TEND', 'DrugBank'], 'pmids': [17640917], 'score': 5}, {'interactionId': 'fc2b2336-7492-4bd6-bc70-0d9a77feb393', 'interactionTypes': [], 'drugName': 'MOTEXAFIN GADOLINIUM', 'drugChemblId': 'CHEMBL3544910', 'sources': ['TdgClinicalTrial', 'DrugBank'], 'pmids': [], 'score': 2}, {'interactionId': 'dbb5c849-7d78-4b41-a794-74f28c1dbecf', 'interactionTypes': [], 'drugName': 'CHEMBL449269', 'drugChemblId': 'CHEMBL449269', 'sources': ['DrugBank'], 'pmids': [10592235], 'score': 2}, {'interactionId': '7436b218-7b03-44e4-aa5f-2370a8ca904f', 'interactionTypes': [], 'drugName': 'FLAVIN ADENINE DINUCLEOTIDE', 'drugChemblId': 'CHEMBL1232653', 'sources': ['DrugBank'], 'pmids': [10592235, 17139284, 17016423], 'score': 4}, {'interactionId': '075b7007-3853-436a-b64b-9e4826dd5818', 'interac

Drugs [{'interactionId': '08806b0a-9307-4924-8f09-00af4b232ef2', 'interactionTypes': ['inhibitor'], 'drugName': 'METHYCLOTHIAZIDE', 'drugChemblId': 'CHEMBL1577', 'sources': ['DrugBank'], 'pmids': [19119014, 10713865, 9546276], 'score': 4}, {'interactionId': 'b8b19f39-7902-424b-ace0-3c5c19fb1af3', 'interactionTypes': ['inhibitor'], 'drugName': 'ETHOXZOLAMIDE', 'drugChemblId': 'CHEMBL18', 'sources': ['TdgClinicalTrial', 'GuideToPharmacologyInteractions', 'ChemblInteractions', 'TEND', 'DrugBank'], 'pmids': [12956733, 10649985, 7929150, 19520577, 6816217, 10995826], 'score': 11}, {'interactionId': '5b0710dd-4217-4b30-987a-3a87ccfa8a81', 'interactionTypes': ['inhibitor'], 'drugName': 'METHOCARBAMOL', 'drugChemblId': 'CHEMBL1201117', 'sources': ['GuideToPharmacologyInteractions', 'ChemblInteractions', 'DrugBank'], 'pmids': [1460006], 'score': 4}, {'interactionId': 'c64fc110-cd22-4c17-9eae-79c3fb6ec24f', 'interactionTypes': ['inhibitor'], 'drugName': 'BENDROFLUMETHIAZIDE', 'drugChemblId': 'CH

Drugs [{'interactionId': 'a0a2f845-195b-4a99-9443-8be8b81ede8c', 'interactionTypes': ['inhibitor'], 'drugName': 'ZONISAMIDE', 'drugChemblId': 'CHEMBL750', 'sources': ['DrugBank'], 'pmids': [17762320, 18537527, 19703035, 18782051, 17582922, 18162396], 'score': 7}, {'interactionId': '429ac158-9736-4dfe-a360-1c21a2366667', 'interactionTypes': ['inhibitor'], 'drugName': 'ELLAGIC ACID', 'drugChemblId': 'CHEMBL6246', 'sources': ['DrugBank'], 'pmids': [20185318], 'score': 2}, {'interactionId': '5d00e809-dca8-41c2-9c03-86bfce0fa19d', 'interactionTypes': ['inhibitor'], 'drugName': 'ETHOXZOLAMIDE', 'drugChemblId': 'CHEMBL18', 'sources': ['ChemblInteractions'], 'pmids': [], 'score': 1}, {'interactionId': '2aa7be29-2275-4b8d-b473-5feef0462249', 'interactionTypes': [], 'drugName': 'ACETAZOLAMIDE', 'drugChemblId': 'CHEMBL20', 'sources': ['NCI'], 'pmids': [1909176], 'score': 2}, {'interactionId': '9bc95854-22f6-4b1d-bb35-f1eab4137e27', 'interactionTypes': [], 'drugName': 'ISOTRETINOIN', 'drugChemblId

...

Gene: GBF1
Gene: GCN1
Gene: GLMN
Gene: GLRX3
Gene: GSPT1
Drugs [{'interactionId': 'c8f3b866-8cf9-41f7-b3e0-42b35ac44d54', 'interactionTypes': [], 'drugName': 'CHEMBL384759', 'drugChemblId': 'CHEMBL384759', 'sources': ['DrugBank'], 'pmids': [10592235], 'score': 2}]
Gene: HDLBP
Gene: HYOU1
Gene: IGBP1
Gene: IGF2BP3
Gene: IRS2
Drugs [{'interactionId': '8eb106e4-cf23-4289-a6c0-fc8ba73167a2', 'interactionTypes': [], 'drugName': 'ASPIRIN', 'drugChemblId': 'CHEMBL25', 'sources': ['NCI'], 'pmids': [18059035], 'score': 2}, {'interactionId': '539b8077-faed-430b-b2f3-7fcfc8f308f8', 'interactionTypes': [], 'drugName': 'DEXAMETHASONE', 'drugChemblId': 'CHEMBL384467', 'sources': ['NCI'], 'pmids': [16943575], 'score': 2}]
Gene: MAGT1
Gene: NMT1
Drugs [{'interactionId': '6821ac58-a5f0-45fb-85d2-239fe57cb3e2', 'interactionTypes': [], 'drugName': 'CHEMBL355497', 'drugChemblId': 'CHEMBL355497', 'sources': ['DrugBank'], 'pmids': [10592235], 'score': 2}]
Gene: NPC1
Gene: NSF
Drugs [{'interactionId': 'caa0e

Drugs [{'interactionId': 'f9c90934-1da8-417d-8c72-3ebc40f5e86c', 'interactionTypes': ['inhibitor'], 'drugName': 'HEXANE', 'drugChemblId': 'CHEMBL15939', 'sources': ['GuideToPharmacologyInteractions'], 'pmids': [], 'score': 1}, {'interactionId': '6376992f-2ce5-4721-b05e-e8657914d697', 'interactionTypes': ['inhibitor'], 'drugName': 'GW620972X', 'drugChemblId': 'CHEMBL233001', 'sources': ['GuideToPharmacologyInteractions'], 'pmids': [], 'score': 1}, {'interactionId': '78f1b608-ea0f-40af-a159-aad64bfe2b46', 'interactionTypes': ['inhibitor'], 'drugName': 'CHEMBL210618', 'drugChemblId': 'CHEMBL210618', 'sources': ['GuideToPharmacologyInteractions'], 'pmids': [], 'score': 1}, {'interactionId': '974f661e-ee5d-46c5-a4cc-2f557b2e9836', 'interactionTypes': ['inhibitor'], 'drugName': 'RGB-286638', 'drugChemblId': 'CHEMBL3545083', 'sources': ['GuideToPharmacologyInteractions'], 'pmids': [], 'score': 1}, {'interactionId': '055fd60f-d488-437e-8c7b-033eb72c7f47', 'interactionTypes': ['inhibitor'], 'dr

Drugs [{'interactionId': '9bd66532-6324-43b0-aec1-75a53c13f003', 'interactionTypes': [], 'drugName': '6-DIAZO-5-OXO-L-NORLEUCINE', 'drugChemblId': 'CHEMBL97485', 'sources': ['NCI'], 'pmids': [7867599], 'score': 2}, {'interactionId': '25470ae8-5c11-478f-9492-275a939cdfd0', 'interactionTypes': [], 'drugName': 'METHOTREXATE', 'drugChemblId': 'CHEMBL34259', 'sources': ['NCI'], 'pmids': [15511236], 'score': 2}, {'interactionId': '87ac33f3-aac6-46e3-84b5-c43874c7a696', 'interactionTypes': [], 'drugName': 'GLUTAMINE', 'drugChemblId': 'CHEMBL930', 'sources': ['NCI'], 'pmids': [9124626, 6146632, 9392478, 11533299, 1174515, 14722097], 'score': 7}, {'interactionId': 'e85243f4-ae36-4379-8e3a-e0a5e6517c48', 'interactionTypes': [], 'drugName': 'DEXAMETHASONE', 'drugChemblId': 'CHEMBL384467', 'sources': ['NCI'], 'pmids': [7904483], 'score': 2}, {'interactionId': 'a0d67bb2-f12f-4ba0-96fb-5859d17e2c37', 'interactionTypes': [], 'drugName': 'L-GLUTAMATE', 'drugChemblId': 'CHEMBL575060', 'sources': ['Drug

Drugs [{'interactionId': '26f9aa35-c529-412d-a7fc-33795930f9af', 'interactionTypes': ['inhibitor'], 'drugName': 'RETASPIMYCIN', 'drugChemblId': 'CHEMBL1184904', 'sources': ['MyCancerGenome', 'TdgClinicalTrial'], 'pmids': [], 'score': 2}, {'interactionId': 'bff67f5e-0eed-4e1d-aba1-8d68f2cd1431', 'interactionTypes': ['inhibitor'], 'drugName': 'NVP-AUY922', 'drugChemblId': 'CHEMBL252164', 'sources': ['MyCancerGenome', 'GuideToPharmacologyInteractions', 'MyCancerGenomeClinicalTrial'], 'pmids': [], 'score': 3}, {'interactionId': '86a47c41-7c3b-40d4-9a13-e5daeb711852', 'interactionTypes': ['inhibitor'], 'drugName': 'GANETESPIB', 'drugChemblId': 'CHEMBL2103879', 'sources': ['MyCancerGenome', 'TdgClinicalTrial', 'ChemblInteractions'], 'pmids': [], 'score': 3}, {'interactionId': 'a23325b5-dbfd-4187-a9fa-8dcc39f1017b', 'interactionTypes': ['inhibitor'], 'drugName': 'TANESPIMYCIN', 'drugChemblId': 'CHEMBL109480', 'sources': ['MyCancerGenome', 'TdgClinicalTrial', 'ChemblInteractions'], 'pmids': []

Drugs [{'interactionId': 'e6d2d057-a8f8-42fc-a310-1265464ad676', 'interactionTypes': ['binder'], 'drugName': 'SPIRONOLACTONE', 'drugChemblId': 'CHEMBL1393', 'sources': ['DrugBank'], 'pmids': [17881754], 'score': 2}, {'interactionId': 'f3c397cd-dab8-4193-8c58-3f6758297d46', 'interactionTypes': [], 'drugName': 'BUSULFAN', 'drugChemblId': 'CHEMBL820', 'sources': ['NCI'], 'pmids': [3692618], 'score': 2}, {'interactionId': 'f26befdd-11e8-4bda-aa9e-61258e002632', 'interactionTypes': [], 'drugName': 'LISINOPRIL', 'drugChemblId': 'CHEMBL1237', 'sources': ['NCI'], 'pmids': [11872207], 'score': 2}, {'interactionId': '7f8b225b-6d29-4644-8b37-a1641475d911', 'interactionTypes': [], 'drugName': 'NORGESTREL', 'drugChemblId': 'CHEMBL2107797', 'sources': ['NCI'], 'pmids': [573191], 'score': 2}, {'interactionId': '51f0ed1a-c5ef-4c19-8adb-97b21d8891ef', 'interactionTypes': [], 'drugName': 'DROLOXIFENE', 'drugChemblId': 'CHEMBL487', 'sources': ['NCI'], 'pmids': [7561649], 'score': 2}]
Gene: SRP72
6.53 s ±

In [107]:
%%timeit
print('UP-REGULATED:\n')
for i, request in enumerate(dgidb_requests_up):
    #print('Genes and drugs for: ' + sig_cols_up[i].columns[0] +':\n')
    json_parse4(request['matchedTerms'])


UP-REGULATED:

Gene: ACBD3
Gene: AVL9
Gene: COG1
Gene: COG4
Gene: GOLPH3L
Gene: IGF2BP3
Gene: RAB3GAP1

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269
[]
Drug: FLAVIN ADENINE DINUCLEOTIDE
[]
Drug: SPERMIDINE
[]
Drug: FOTEMUSTINE
[]


Gene: ACBD3
Gene: GOLPH3L
Gene: GSDME
Gene: IGF2BP3

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269
[]
Drug: FLAVIN ADENINE DINUCLEOTIDE
[]
Drug: SPERMIDINE
[]
Drug: FOTEMUSTINE
[]


Gene: AGL
Gene: ATG5
Gene: CTPS2
Gene: DCAF7

Gene: HDAC4
Drug: TASQUINIMOD
['allosteric modulator']
Drug: BELINOSTAT
['inhibitor']
Drug: CUDC-101
['inhibitor']
Drug: DACINOSTAT
['inhibitor']
Drug: CHEMBL1213492
['inhibitor']
Drug: PANOBINOSTAT
['inhibitor']
Drug: QUISINOSTAT
['inhibitor']
Drug: ROMIDEPSIN
['inhibitor']
Drug: SCRIPTAID
['inhibitor']
Drug: CHEMBL3110004
['inhibitor']
Drug: TRICHOSTATIN
['inhibitor']
Drug: CHEMBL1851943
['inhibitor']
Drug: PANOBINOSTAT LACTATE

Gene: PSMC6
Drug: OPROZOMIB
['inhibitor']
Drug: CARFILZOMIB
['inhibitor']
Drug: IXAZOMIB CITRATE
['inhibitor']
Drug: BORTEZOMIB
['inhibitor']



Gene: PSME4
Drug: BORTEZOMIB
['inhibitor']
Drug: CARFILZOMIB
['inhibitor']


Gene: PTBP1
Gene: PUS7L
Gene: RAD23A
Gene: RBM4
Gene: RFESD
Gene: RMND5A
Gene: RNF123
Gene: RNF135
Gene: RNPS1
Gene: RPL22
Gene: SAV1
Gene: SLC4A1
Gene: SLC9A6
Gene: SPRY1
Gene: SPTA1
Gene: SRPK1
Gene: STARD8
Gene: STN1
Gene: STOM
Gene: TPST2
Gene: TTLL4
Gene: UBQLN1
Gene: USP42

Gene: WRAP53
Drug: CARBOQUONE
[]
Drug: FLUOROURACIL
[]
Drug: CYCLOPHOSPHAMIDE
[]


Gene: YOD1
Gene: ZBTB33
Gene: BMP1
Gene: CALCOCO1
Gene: CMAS
Gene: CSE1L
Gene: DLG5
Gene: EIF3D
Gene: FLNC
Gene: NPC1
Gene: NUDC
Gene: RBCK1
Gene: SFN
Gene: SPICE1

Gene: TFPI2
Drug: HEPARIN
[]


Gene: ACBD3
Gene: GOLPH3L

Gene: HSP90AB1
Drug: RETASPIMYCIN
['inhibitor']
Drug: NVP-AUY922
['inhibitor']
Drug: GANETESPIB
['inhibitor']
Drug: TANESPIMYCIN
['inhibitor']
Drug: RETASPIMYCIN HYDROCHLORIDE (CHEMBL377559)


['multitarget', 'antagonist', 'inhibitor']
Drug: TAMOXIFEN
[]
Drug: ESTRONE SODIUM SULFATE
[]
Drug: THROMBOMODULIN ALFA
[]
Drug: ELTROMBOPAG
[]
Drug: THROMBIN
[]


Gene: GNPTG
Gene: MRFAP1
Gene: SCYL3

Gene: SHBG
Drug: SPIRONOLACTONE
['binder']
Drug: BUSULFAN
[]
Drug: LISINOPRIL
[]
Drug: NORGESTREL
[]
Drug: DROLOXIFENE
[]


Gene: SRP72
UP-REGULATED:

Gene: ACBD3
Gene: AVL9
Gene: COG1
Gene: COG4
Gene: GOLPH3L
Gene: IGF2BP3
Gene: RAB3GAP1

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269
[]
Drug: FLAVIN ADENINE DINUCLEOTIDE
[]
Drug: SPERMIDINE
[]
Drug: FOTEMUSTINE
[]


Gene: ACBD3
Gene: GOLPH3L
Gene: GSDME
Gene: IGF2BP3

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269
[]
Drug: FLAVIN ADENINE DINUCLEOTIDE
[]
Drug: SPERMIDINE
[]
Drug: FOTEMUSTINE
[]


Gene: AGL
Gene: ATG5
Gene: CTPS2
Gene: DCAF7

Gene: HDAC4
Drug: TASQUINIMOD
['allosteric modulator']
Drug: BELINOSTAT
['inhibitor']
Drug: CUD

[]
Drug: CHEMBL522579
[]
Drug: CHEMBL257341
[]
Drug: CHEMBL195450
[]


Gene: MCRS1

Gene: MTRR
Drug: CYANOCOBALAMIN
['cofactor']
Drug: METHIONINE
['product of']
Drug: HYDROXOCOBALAMIN
['other']
Drug: VITAMIN B12
[]
Drug: TEGAFUR
[]


Gene: NAP1L1
Gene: NBEAL2
Gene: PLEKHG4B
Gene: POMGNT1
Gene: PON3

Gene: PRDX2
Drug: CHEMBL1160508
[]



Gene: PSMC6
Drug: OPROZOMIB
['inhibitor']
Drug: CARFILZOMIB
['inhibitor']
Drug: IXAZOMIB CITRATE
['inhibitor']
Drug: BORTEZOMIB
['inhibitor']



Gene: PSME4
Drug: BORTEZOMIB
['inhibitor']
Drug: CARFILZOMIB
['inhibitor']


Gene: PTBP1
Gene: PUS7L
Gene: RAD23A
Gene: RBM4
Gene: RFESD
Gene: RMND5A
Gene: RNF123
Gene: RNF135
Gene: RNPS1
Gene: RPL22
Gene: SAV1
Gene: SLC4A1
Gene: SLC9A6
Gene: SPRY1
Gene: SPTA1
Gene: SRPK1
Gene: STARD8
Gene: STN1
Gene: STOM
Gene: TPST2
Gene: TTLL4
Gene: UBQLN1
Gene: USP42

Gene: WRAP53
Drug: CARBOQUONE
[]
Drug: FLUOROURACIL
[]
Drug: CYCLOPHOSPHAMIDE
[]


Gene: YOD1
Gene: ZBTB33
Gene: BMP1
Gene: CALCOCO1
Gene: CMAS
Gene: CSE1L
Ge

['antagonist']


Gene: PPP1R37
Gene: SLC38A10
Gene: STK39
Gene: TMBIM1

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269
[]
Drug: FLAVIN ADENINE DINUCLEOTIDE
[]
Drug: SPERMIDINE
[]
Drug: FOTEMUSTINE
[]



Gene: CPN1
Drug: PYRAZINAMIDE
['inhibitor']



Gene: F5
Drug: DROTRECOGIN ALFA (ACTIVATED)
['multitarget', 'antagonist', 'inhibitor']
Drug: TAMOXIFEN
[]
Drug: ESTRONE SODIUM SULFATE
[]
Drug: THROMBOMODULIN ALFA
[]
Drug: ELTROMBOPAG
[]
Drug: THROMBIN
[]


Gene: GNPTG
Gene: MRFAP1
Gene: SCYL3

Gene: SHBG
Drug: SPIRONOLACTONE
['binder']
Drug: BUSULFAN
[]
Drug: LISINOPRIL
[]
Drug: NORGESTREL
[]
Drug: DROLOXIFENE
[]


Gene: SRP72
UP-REGULATED:

Gene: ACBD3
Gene: AVL9
Gene: COG1
Gene: COG4
Gene: GOLPH3L
Gene: IGF2BP3
Gene: RAB3GAP1

Gene: TXNRD1
Drug: ARSENIC TRIOXIDE
['inhibitor']
Drug: MOTEXAFIN GADOLINIUM
[]
Drug: CHEMBL449269
[]
Drug: FLAVIN ADENINE DINUCLEOTIDE
[]
Drug: SPERMIDINE
[]
Drug: FOTEMUSTINE
[]


Gene: ACBD3
Gene: GOLPH3L
Gene

...

Gene: C8ORF59
Gene: FAM107B

Gene: MARK2
Drug: ACETAMINOPHEN
[]


Gene: TEX2
Gene: C12ORF29

Gene: CALR
Drug: TRETINOIN
[]
Drug: GENTAMICIN
[]
Drug: TENECTEPLASE
[]
Drug: ETRETINATE
[]
Drug: MELATONIN
[]
Drug: OLTIPRAZ
[]



Gene: CD44
Drug: HYALURONIC ACID
[]
Drug: STAUROSPORINE
[]
Drug: INTERFERON GAMA-1B
[]
Drug: MYCOPHENOLIC ACID
[]
Drug: GENTAMICIN
[]
Drug: MOMETASONE FUROATE
[]


Gene: DST
Gene: GALNT2

Gene: GLS
Drug: 6-DIAZO-5-OXO-L-NORLEUCINE
[]
Drug: METHOTREXATE
[]
Drug: GLUTAMINE
[]
Drug: DEXAMETHASONE
[]
Drug: L-GLUTAMATE
[]


Gene: GORASP2

Gene: HSP90AB1
Drug: RETASPIMYCIN
['inhibitor']
Drug: NVP-AUY922
['inhibitor']
Drug: GANETESPIB
['inhibitor']
Drug: TANESPIMYCIN
['inhibitor']
Drug: RETASPIMYCIN HYDROCHLORIDE (CHEMBL377559)
['inhibitor']
Drug: ONALESPIB
[]
Drug: CHEMBL399530
[]
Drug: ALVESPIMYCIN
[]
Drug: RADICICOL
[]
Drug: GELDANAMYCIN
[]
Drug: CHEMBL383189
[]
Drug: CHEMBL2204502
[]
Drug: CHEMBL553939
[]
Drug: CHEMBL113690
[]
Drug: CHEMBL2170470
[]


Gene: IGF2BP3

G

['inhibitor']
Drug: LEO-15520
['inhibitor']
Drug: R-1487 (CHEMBL1766582)
['inhibitor']
Drug: Ro-3201195
['inhibitor']
Drug: RWJ-67657
['inhibitor']
Drug: SC-80036
['inhibitor']
Drug: SCIO-323
['inhibitor']
Drug: SD-0006
['inhibitor']
Drug: TA-5493
['inhibitor']
Drug: ARRY-797
['inhibitor']
Drug: SEMAPIMOD
['inhibitor']
Drug: PG-760564
['inhibitor']
Drug: PS-516895
['inhibitor']
Drug: FX-005
['inhibitor']
Drug: AMG-548
['inhibitor']
Drug: AVE-9940
['inhibitor']
Drug: SB-85635
['inhibitor']
Drug: KC-706
['inhibitor']
Drug: PAMAPIMOD
['inhibitor']
Drug: Pexmetinib
['inhibitor']
Drug: LY-3007113
['inhibitor']
Drug: AZD-7624
['inhibitor']
Drug: CHEMBL305178
[]
Drug: DOXORUBICIN
[]
Drug: VASOPRESSIN
[]
Drug: EPOETIN BETA
[]
Drug: HYDROGEN PEROXIDE
[]
Drug: CHEMBL194009
[]
Drug: CHEMBL494072
[]
Drug: CHEMBL194322
[]
Drug: CHEMBL237127
[]
Drug: RALIMETINIB MESYLATE
[]
Drug: SULINDAC SULFIDE
[]
Drug: CHEMBL87277
[]
Drug: CHEMBL193156
[]
Drug: 2-CHLOROPHENOL
[]
Drug: CHEMBL564912
[]
Drug: CHEMBL

Drug: GLUTAMINE
[]
Drug: DEXAMETHASONE
[]
Drug: L-GLUTAMATE
[]


Gene: GORASP2
Gene: IGF2BP3

Gene: IRS2
Drug: ASPIRIN
[]
Drug: DEXAMETHASONE
[]


Gene: MAGED1
Gene: NUDC
Gene: ORC5
Gene: PDIA6
Gene: PKP3
Gene: POP1

Gene: PYCR2
Drug: CHEMBL1161866
[]
Drug: L-PROLINE
[]


Gene: RRBP1
Gene: SLC38A10

Gene: TARS
Drug: THREONINE
[]


Gene: TNPO3
Gene: C8ORF59
Gene: FAM107B

Gene: MARK2
Drug: ACETAMINOPHEN
[]


Gene: TEX2
Gene: C12ORF29

Gene: CALR
Drug: TRETINOIN
[]
Drug: GENTAMICIN
[]
Drug: TENECTEPLASE
[]
Drug: ETRETINATE
[]
Drug: MELATONIN
[]
Drug: OLTIPRAZ
[]



Gene: CD44
Drug: HYALURONIC ACID
[]
Drug: STAUROSPORINE
[]
Drug: INTERFERON GAMA-1B
[]
Drug: MYCOPHENOLIC ACID
[]
Drug: GENTAMICIN
[]
Drug: MOMETASONE FUROATE
[]


Gene: DST
Gene: GALNT2

Gene: GLS
Drug: 6-DIAZO-5-OXO-L-NORLEUCINE
[]
Drug: METHOTREXATE
[]
Drug: GLUTAMINE
[]
Drug: DEXAMETHASONE
[]
Drug: L-GLUTAMATE
[]


Gene: GORASP2

Gene: HSP90AB1
Drug: RETASPIMYCIN
['inhibitor']
Drug: NVP-AUY922
['inhibitor']
Drug: GANETESPI

Drug: PH-797804
['inhibitor']
Drug: CHEMBL306380
['inhibitor']
Drug: RALIMETINIB
['inhibitor']
Drug: SB-202190
['inhibitor']
Drug: SB-203580
['inhibitor']
Drug: SB-220025
['inhibitor']
Drug: CHEMBL313417
['inhibitor']
Drug: TAK-715
['inhibitor']
Drug: TALMAPIMOD
['inhibitor']
Drug: VX-702
['inhibitor']
Drug: VX-745
['inhibitor']
Drug: GSK-610677
['inhibitor']
Drug: LEO-15520
['inhibitor']
Drug: R-1487 (CHEMBL1766582)
['inhibitor']
Drug: Ro-3201195
['inhibitor']
Drug: RWJ-67657
['inhibitor']
Drug: SC-80036
['inhibitor']
Drug: SCIO-323
['inhibitor']
Drug: SD-0006
['inhibitor']
Drug: TA-5493
['inhibitor']
Drug: ARRY-797
['inhibitor']
Drug: SEMAPIMOD
['inhibitor']
Drug: PG-760564
['inhibitor']
Drug: PS-516895
['inhibitor']
Drug: FX-005
['inhibitor']
Drug: AMG-548
['inhibitor']
Drug: AVE-9940
['inhibitor']
Drug: SB-85635
['inhibitor']
Drug: KC-706
['inhibitor']
Drug: PAMAPIMOD
['inhibitor']
Drug: Pexmetinib
['inhibitor']
Drug: LY-3007113
['inhibitor']
Drug: AZD-7624
['inhibitor']
Drug: CHEM

Drug: TRETINOIN
[]
Drug: GENTAMICIN
[]
Drug: TENECTEPLASE
[]
Drug: ETRETINATE
[]
Drug: MELATONIN
[]
Drug: OLTIPRAZ
[]


Gene: GALNT2

Gene: GLS
Drug: 6-DIAZO-5-OXO-L-NORLEUCINE
[]
Drug: METHOTREXATE
[]
Drug: GLUTAMINE
[]
Drug: DEXAMETHASONE
[]
Drug: L-GLUTAMATE
[]


Gene: GORASP2
Gene: IGF2BP3

Gene: IRS2
Drug: ASPIRIN
[]
Drug: DEXAMETHASONE
[]


Gene: MAGED1
Gene: NUDC
Gene: ORC5
Gene: PDIA6
Gene: PKP3
Gene: POP1

Gene: PYCR2
Drug: CHEMBL1161866
[]
Drug: L-PROLINE
[]


Gene: RRBP1
Gene: SLC38A10

Gene: TARS
Drug: THREONINE
[]


Gene: TNPO3
Gene: C8ORF59
Gene: FAM107B

Gene: MARK2
Drug: ACETAMINOPHEN
[]


Gene: TEX2
Gene: C12ORF29

Gene: CALR
Drug: TRETINOIN
[]
Drug: GENTAMICIN
[]
Drug: TENECTEPLASE
[]
Drug: ETRETINATE
[]
Drug: MELATONIN
[]
Drug: OLTIPRAZ
[]



Gene: CD44
Drug: HYALURONIC ACID
[]
Drug: STAUROSPORINE
[]
Drug: INTERFERON GAMA-1B
[]
Drug: MYCOPHENOLIC ACID
[]
Drug: GENTAMICIN
[]
Drug: MOMETASONE FUROATE
[]


Gene: DST
Gene: GALNT2

Gene: GLS
Drug: 6-DIAZO-5-OXO-L-NORLEUCI

In [59]:
'''#%%timeit
print('DOWN-REGULATED:\n')
for i, request in enumerate(dgidb_requests_down):
    print('Genes and drugs for: ' + sig_cols_down[i].columns[0] +':\n')
    drug_dict_down = bf.dgidb_json_parse(request)
'''

"#%%timeit\nprint('DOWN-REGULATED:\n')\nfor i, request in enumerate(dgidb_requests_down):\n    print('Genes and drugs for: ' + sig_cols_down[i].columns[0] +':\n')\n    drug_dict_down = bf.dgidb_json_parse(request)\n"